# DESCARGAR VIDEO YOUTUBE, CON SONIDO Y AÑADIRLE VOZ EN OFF CON TRADUCCIÓN

In [2]:
import os
# import openai
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

# Todas las variables de entorno se cargan desde el archivo .env
DEEPL_API = os.getenv('DEEPL_API')

In [3]:
import os

# Establecer las variables de entorno
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.3'
os.environ['LD_LIBRARY_PATH'] = os.path.join(os.environ['CUDA_HOME'], 'targets/x86_64-linux/lib/stubs') + ':' + os.environ.get('LD_LIBRARY_PATH', '')
os.environ['LD_LIBRARY_PATH'] = os.path.join(os.environ['CUDA_HOME'], 'lib64') + ':' + os.environ.get('LD_LIBRARY_PATH', '')
os.environ['PATH'] = os.path.join(os.environ['CUDA_HOME'], 'bin') + ':' + os.environ.get('PATH', '')

os.environ['CUDNN_HOME'] = '/home/javier/cosas/cudnn-linux-x86_64-8.8.1.3_cuda11-archive'
os.environ['LD_LIBRARY_PATH'] = os.path.join(os.environ['CUDNN_HOME'], 'lib64') + ':' + os.environ.get('LD_LIBRARY_PATH', '')
os.environ['CPATH'] = os.path.join(os.environ['CUDNN_HOME'], 'include') + ':' + os.environ.get('CPATH', '')
os.environ['LIBRARY_PATH'] = os.path.join(os.environ['CUDNN_HOME'], 'lib64') + ':' + os.environ.get('LIBRARY_PATH', '')

# Verificar las variables de entorno
for var in ['CUDA_HOME', 'LD_LIBRARY_PATH', 'PATH', 'CUDNN_HOME', 'CPATH', 'LIBRARY_PATH']:
    print(f'{var}: {os.environ.get(var)}')

CUDA_HOME: /usr/local/cuda-12.3
LD_LIBRARY_PATH: /home/javier/cosas/cudnn-linux-x86_64-8.8.1.3_cuda11-archive/lib64:/usr/local/cuda-12.3/lib64:/usr/local/cuda-12.3/targets/x86_64-linux/lib/stubs:
PATH: /usr/local/cuda-12.3/bin:/home/javier/miniconda3/envs/mistral/bin:/home/javier/.vscode-server/bin/6c3e3dba23e8fadc360aed75ce363ba185c49794/bin/remote-cli:/home/javier/miniconda3/condabin:/usr/lib/wsl/lib:/usr/local/cuda-12.3/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
CUDNN_HOME: /home/javier/cosas/cudnn-linux-x86_64-8.8.1.3_cuda11-archive
CPATH: /home/javier/cosas/cudnn-linux-x86_64-8.8.1.3_cuda11-archive/include:
LIBRARY_PATH: /home/javier/cosas/cudnn-linux-x86_64-8.8.1.3_cuda11-archive/lib64:


In [4]:
import importlib
import sys
# he creado un modulo de herramientas con los enlaces simbólicos a los modulos que uso en los notebooks
# insertar carpeta si no esta en sys.path
if not '../herramientas/' in sys.path:
    sys.path.insert(0, '../herramientas/')
import tunel_ssh_socks

importlib.reload(tunel_ssh_socks)

# ver todas las carpetas de sys.path y su orden de busqueda
for path in sys.path:
    print(path)

../herramientas/
/mnt/d/DeveloperIA/video-translator
/home/javier/miniconda3/envs/mistral/lib/python310.zip
/home/javier/miniconda3/envs/mistral/lib/python3.10
/home/javier/miniconda3/envs/mistral/lib/python3.10/lib-dynload

/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages


In [5]:
import tunel_ssh_socks as tunel
tunel.esta_activo_tunel_ssh()

False

In [5]:
import os

media_path = os.path.join(os.getcwd(), 'media')
# si no existe la carpeta media, la creamos
if not os.path.exists(media_path):
    os.makedirs(media_path)

# nombre del video a crear dentro de la carpeta media
path_video = os.path.join(media_path, 'Lesson_4_Practical_Deep_Learning_for_Coders_2022.mp4')

In [6]:
import os

output_audio = path_video.replace(".mp4", ".mp3")
carpeta_transcripcion = os.path.join(os.getcwd(), "audio_transcription")
carpeta_transcripcion_es = os.path.join(os.getcwd(), "audio_transcription_es")
#coger solo nombre del video sin path
nombre_video = os.path.basename(path_video)
fichero_transcripcion = nombre_video.replace(".mp4", ".vtt")
fichero_csv = nombre_video.replace(".mp4", ".csv")
path_csv = os.path.join(carpeta_transcripcion, fichero_csv)
nombre_transcripcion = os.path.join(carpeta_transcripcion, fichero_transcripcion)
nombre_transcripcion_es = os.path.join(carpeta_transcripcion_es, fichero_transcripcion)
nombre_transcripcion_adaptada = os.path.join(carpeta_transcripcion, nombre_video.replace(".mp4", "_adaptados.vtt"))
#se traducirá la transcripción una vez se haya adaptado para mejorar la traducción
nombre_transcripcion_trad = os.path.join(carpeta_transcripcion, nombre_video.replace(".mp4", "_traducidos.vtt"))
#se adaptará la transcripción traducida para que sea más corta para el T2S
nombre_transcripcion_lista_t2s = os.path.join(carpeta_transcripcion, nombre_video.replace(".mp4", "_lista_t2s.vtt"))

video_transcrito_name = nombre_video.replace(".mp4", "_transcribed.mp4")



print(carpeta_transcripcion)
print("video:", nombre_video)
print("audio:", output_audio)
print("transcripción:", nombre_transcripcion)
print("transcripción: raducida",nombre_transcripcion_trad)
print("transcripción adaptada:",nombre_transcripcion_adaptada)
print("transcripción lista para t2s:",nombre_transcripcion_lista_t2s)
# print(nombre_transcripcion_es)
print("media_path:", media_path)
print("nombre video transcrito", video_transcrito_name)
!export TF_ENABLE_ONEDNN_OPTS=0 #para desactivar el optimizador que puede dar problemas de accuracy


/mnt/d/DeveloperIA/video-translator/audio_transcription
video: Lesson_4_Practical_Deep_Learning_for_Coders_2022.mp4
audio: /mnt/d/DeveloperIA/video-translator/media/Lesson_4_Practical_Deep_Learning_for_Coders_2022.mp3
transcripción: /mnt/d/DeveloperIA/video-translator/audio_transcription/Lesson_4_Practical_Deep_Learning_for_Coders_2022.vtt
transcripción: raducida /mnt/d/DeveloperIA/video-translator/audio_transcription/Lesson_4_Practical_Deep_Learning_for_Coders_2022_traducidos.vtt
transcripción adaptada: /mnt/d/DeveloperIA/video-translator/audio_transcription/Lesson_4_Practical_Deep_Learning_for_Coders_2022_adaptados.vtt
transcripción lista para t2s: /mnt/d/DeveloperIA/video-translator/audio_transcription/Lesson_4_Practical_Deep_Learning_for_Coders_2022_lista_t2s.vtt
media_path: /mnt/d/DeveloperIA/video-translator/media
nombre video transcrito Lesson_4_Practical_Deep_Learning_for_Coders_2022_transcribed.mp4


In [ ]:
!cat {nombre_transcripcion}

In [4]:
import subprocess
import pkg_resources

try:
    # Intenta importar el paquete
    pkg_resources.get_distribution('pytube')
    print('pytube ya está instalado.')
except pkg_resources.DistributionNotFound:
    # Si el paquete no está instalado, instálalo
    print('Instalando pytube...')
    subprocess.run(['pip', 'install', 'pytube'])
    print('pytube instalado exitosamente.')

# otra forma
# try:
#     import pytube
# except ImportError:
#     !pip install pytube


pytube ya está instalado.


In [38]:
from pytube import YouTube


def Download(link):
    youtubeObject = YouTube(link)
    youtubeObject = youtubeObject.streams.get_highest_resolution()
    print("Downloading...")
    try:
        youtubeObject.download(filename=path_video)
    except:
        print("An error has occurred")
    print("Download is completed successfully")
    

link = input("Enter the YouTube video URL: ")
Download(link)


Downloading...
Download is completed successfully


In [ ]:
# %%capture
# !apt update
# !pip install git+https://github.com/openai/whisper.git
# !apt install ffmpeg

# Separamos audio de video y pasamos Whisper para sacar la transcripción

In [18]:
import os

#SEPARAMOS EL AUDIO EN OTRO FICHERO (LUEGO LO JUNTAREMOS OTRA VEZ)
!ffmpeg -y -i "$path_video" -c:a libmp3lame -qscale:a 1 "$output_audio"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### Transcripción de audio a texto (ingles a ingles)

Hacemos la transcripción

In [40]:
# import whisper

#generamos la transcripción con whisper
!whisper  {output_audio} --task transcribe --model large --verbose False --output_dir {carpeta_transcripcion}

100%|█████████████████████████████████████| 2.88G/2.88G [00:44<00:00, 69.9MiB/s]
Detected language: English
100%|██████████████████████████████| 392500/392500 [08:21<00:00, 782.35frames/s]


### Transcripción de audio a texto (desde cualquier idioma a ingles)

Si quisieramos traducir desde otro idioma distinto del inglés como el frances, alemán, chino, etc, podríamos primero usar la traducción automática de la transcripción de Whisper al inglés y luego traducir esta transcripción al español con un modelo traductor del ingles al español

In [32]:
# import whisper

#generamos la transcripción con whisper
!whisper  {output_audio} --task translate --model large --verbose False --output_dir {carpeta_transcripcion}

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)


Detected language: French
100%|████████████████████████████████| 73008/73008 [02:03<00:00, 593.31frames/s]


### Traducción de texto a audio (desde inglés al español) (se podría hacer desde el inglés a cualquier idioma --languaje="idioma")

También se puede hacer transcripción al mismo tiempo que traducción desde inglés a español (es más rápida pero la traducción inversa no está demasiado conseguida, para mayor calidad usar transcripción normal y luego convertir al español usando modelo de traducción):

In [41]:
import whisper

#generamos la transcripción con whisper
!whisper  {output_audio} --task transcribe --model large-v2 --language="Spanish" --verbose False --output_dir {carpeta_transcripcion_es}


100%|██████████████████████████████| 392500/392500 [10:05<00:00, 648.23frames/s]


## Mezclar video con transcripción

### Mezclar video con transcripción en ingles (sin traducir)

In [ ]:
!ffmpeg -y -i {path_video} -i {output_audio} -i {nombre_transcripcion} -map 0:v -map 1:a -map 2 -metadata:s:s:0 language=eng -c:v copy -c:a aac -c:s mov_text {video_transcrito_name}


### Mezclar video con transcripción en español (traducida)

In [ ]:
!ffmpeg -y -i {path_video} -i {output_audio} -i {nombre_transcripcion_es} -map 0:v -map 1:a -map 2 -metadata:s:s:0 language=spa -c:v copy -c:a aac -c:s mov_text {video_transcrito_name}

### Mezclar video con las 2 pistas.

In [20]:
!ffmpeg -y -i {path_video} -i {output_audio} -i {nombre_transcripcion} -i {nombre_transcripcion_es} -map 0:v -map 1:a -map 2 -map 3 -metadata:s:s:0 language=eng -metadata:s:s:1 language=spa -c:v copy -c:a aac -c:s mov_text "$video_transcrito_name"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# PARA TRADUCIR .VTT AL Español podemos usar un modelo de hugginface de tipo Helsinki-NLP:

In [8]:
from transformers import MarianMTModel, MarianTokenizer

def translate_text_to_spanish(text):
    model_name = 'Helsinki-NLP/opus-mt-en-es'  # Modelo para traducir de inglés a español
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    
    tokens = tokenizer(text, return_tensors='pt', padding=True)
    translation = model.generate(**tokens)
    decoded = tokenizer.decode(translation[0], skip_special_tokens=True)
    
    return decoded

# text = "This is the story of Diego de la Vega and of how he became the legendary Zorro. At last I am able to reveal his identity, which for so many years we kept secret, though I do so with some unease, since a blank page is more intimidating to me than the naked swords of Moncada’s men."
text = """Science has discovered two, exactly two ways to memorize things.$Mnemonic cues and spaced repetition.$When I say mnemonic cue, I mean an association, like a story.$
"""

# text = """When you do this properly, what happens is that you see the symbol, $ 
# and then that causes you to recall the story that you told yourself or whatever, $
# and the story causes you to recall the other piece of information that you were trying to attach to that symbol. $
# """
translated_text = translate_text_to_spanish(text)
print(f"Traducción: {translated_text}")

Traducción: La ciencia ha descubierto dos, exactamente dos maneras de memorizar las cosas. $ señales mnemónicas y repetición espaciada. $ Cuando digo señal mnemónica, me refiero a una asociación, como una historia. $


In [86]:
# intermedio1 = "".join(translated_text)
# intermedio1.split(" $ ")
# # quitar último $
# intermedio1 = intermedio1.replace(" $", " ")
# print(intermedio1)

La ciencia ha descubierto dos, exactamente dos maneras de memorizar las cosas.  señales mnemónicas y repetición espaciada.  Cuando digo señal mnemónica, me refiero a una asociación, como una historia. 


Para traducir del español al inglés se puede hacer directamente desde Whisper. Este traductor de inglés a español sería útil, sobre todo, para traducción de texto a texto.

In [2]:
from transformers import MarianMTModel, MarianTokenizer

def translate_text_to_english(text):
    model_name = 'Helsinki-NLP/opus-mt-es-en'  # Modelo para traducir de español a inglés
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    
    tokens = tokenizer(text, return_tensors='pt', padding=True)
    translation = model.generate(**tokens)
    decoded = tokenizer.decode(translation[0], skip_special_tokens=True)
    
    return decoded

text = "Esta es la historia de Diego de la Vega y de cómo se convirtió en el legendario Zorro. Por fin soy capaz de revelar su identidad, que durante tantos años mantuvimos en secreto, aunque lo hago con cierta inquietud, ya que una página en blanco es más intimidante para mí que las espadas desnudas de los hombres de Moncada."
text = "Mientras ella seguía presionando con su tacón sobre mi pecho, apoyando todo su peso e infligiendome un dolor que me paralizaba. con la bota medio abierta, dejando ver parte de la pantorrilla, Sacó una pequeña navaja , y dijo: 'no es nada personal, solo negocios, si te mato me quedo con todas tus cosas' mientra me degollaba con dificultad. Después limpió la navaja en mi camisa blanca riendo y volvió a meterse la navaja en su bota. Luego mientras iba subiendo su cremallera yo ya agonizaba pero ese sonido de esa cremallera me hizo sentir caliente hasta mi último suspiro."
translated_text = translate_text_to_english(text)
print(f"Traducción: {translated_text}")


Traducción: While she kept pushing her heel over my chest, supporting all her weight and inflicting a pain that paralyzed me. with her boot half open, letting me see part of the calf, she pulled out a little knife, and said: 'it's nothing personal, just business, if I kill you I stay with all your things' while she killed me with difficulty. Then she cleaned the knife in my white shirt laughing and put the knife back in her boot. Then as I went up her zipper I was already dying but that sound of that zipper made me feel hot until my last sigh.


### Traducción simple: linea por linea:

In [5]:
# import re

# def traducir(tipo_traductor, text, origen="eng", destino="spa"):
#     if tipo_traductor == "helsinki":
#         if origen == "eng":
#             translated_text = translate_text_to_spanish(text)
#         else:
#             translated_text = translate_text_to_english(text)
#         return translated_text

# def traducir_subtitulos(tipo_traductor, input_file, output_file, origen="eng", destino="spa"):
#     with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
#         sentence = ""
#         segments_count = 0
#         for line in infile:
#             if re.match(r'^\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}\.\d{3}', line) or line.strip() == "" or line.strip() == "WEBVTT":
#                 outfile.write(line)
#             else:
#                 print("line:", line.strip())           
#                 translated_sentence = traducir(tipo_traductor, line.strip(), origen, destino)
#                 print("translated_sentence:", translated_sentence, "\n")
#                 outfile.write(translated_sentence + '\n')


### Mejor traducción: agrupar lineas para tener mejor contexto. (y también traduzca más rápido)

Las lineas se agruparán así:
```bash
"""Science has discovered two, exactly two ways to memorize things. $ 
Mnemonic cues and spaced repetition. $
When I say mnemonic cue, I mean an association, like a story. $


Traducción: Cuando haces esto correctamente, lo que sucede es que ves el símbolo, $ y luego eso te hace recordar la historia que te contaste a ti mismo o lo que sea, $ y la historia te hace recordar la otra pieza de información que estabas tratando de adjuntar a ese símbolo. $
```


Y después se traducirán y volverán a meterse cada una traducida con sus respectivas cabeceras de tiempo.

Aquí el código para hacer esa agrupación mejorando la traducción simple. Lo que hará será meter en cada iteración la lógica que agrupe mínimo N lineas hasta que encutre un ".", las separe por $ y las traduzca, luego separará por $ y las pondrá junto con sus respectivas cabeceras de tiempo.

In [1]:
# traducir con DeepL
%pip install deepl

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [14]:
def traducir_deepl(text, origen="eng", destino="spa"):
    import deepl

    translator = deepl.Translator(DEEPL_API)
    if origen=="eng":
        result = translator.translate_text(text, target_lang="ES")
        return result.text
    elif origen=="spa":
        result = translator.translate_text(text, target_lang="EN")
        return result.text
    else:
        return "idioma origen no soportado"


In [79]:
def traducir_google_translate(text, origen="eng", destino="spa"):
    import tunel_ssh_socks as tunel
    import random
    import time
    from googletrans import Translator
    
    #Configuración del servidor SSH y credenciales
    ssh_host = "74.208.163.90"  # IP del servidor SSH
    local_port = 5559               # Puerto SSH
    username = "root"     
    path_to_private_key = "./.ssh/tunel"  # Ruta clave privada SSH

    # crear tunel si no existe
    if not tunel.esta_activo_tunel_ssh():
        tunel.crear_tunel_ssh(local_port, username, ssh_host, path_to_private_key)
        tunel.crear_proxy_socks(local_port)
        print("esta_activo:", tunel.esta_activo_tunel_ssh())
        tunel.obtener_informacion_ip()

    # traducción
    traductor = Translator()
    if origen=="eng":
        # dormir entre 1000 y 1500 ms para evitar que google detecte que somos un bot
        time.sleep(random.uniform(1, 1.5))
        return traductor.translate(text, src='en', dest='es').text
    elif origen=="spa":
        time.sleep(random.uniform(1, 1.5))
        return traductor.translate(text, src='es', dest='en').text
    else:
        return "No traducido"

In [36]:
import re

def traducir(tipo_traductor, text, origen="eng", destino="spa"):
    if tipo_traductor == "helsinki":
        if origen == "eng":
            translated_text = translate_text_to_spanish(text)
        else:
            translated_text = translate_text_to_english(text)
        return translated_text
    elif tipo_traductor == "google":
        translated_text = traducir_google_translate(text, origen, destino)
        return translated_text
    elif tipo_traductor == "deepl":
        translated_text = traducir_deepl(text, origen, destino)
        return translated_text

    

SEPARADOR = "#"
def parchearSeparadoresPerdidos(texto_original, texto_traducido):
 
    num_parches = texto_original.count(SEPARADOR) - texto_traducido.count(SEPARADOR)
    # metemos los textos en listas para comparar tramo a tramo
    lista_texto_original = texto_original.split(SEPARADOR)
    lista_texto_traducido = texto_traducido.split(SEPARADOR)
    # vemos en que tramo hay un 70% más de palabras que en el original
    for i in range(len(lista_texto_traducido)):
        lista_palabras_tramo_traducido = lista_texto_traducido[i].split()
        if len(lista_palabras_tramo_traducido) > 10 and len(lista_palabras_tramo_traducido) > len(lista_texto_original[i].split()) * 1.6:
            # añadimos separador en la mitad de la frase
            mitad = len(lista_palabras_tramo_traducido) // 2
            lista_texto_traducido[i] = " ".join([" ".join(lista_texto_traducido[i].split()[:mitad]), SEPARADOR, " ".join(lista_texto_traducido[i].split()[mitad:])])
            num_parches -= 1
            if num_parches == 0:
                break
    # unimos los textos traducidos
    texto_traducido = SEPARADOR.join(lista_texto_traducido)
    return texto_traducido


def traducir_subtitulos(tipo_traductor, input_file, output_file, origen="eng", destino="spa"):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        csv_buffer = ""
        primera_linea = True
        max_paquetes = 16 #6 #número de líneas que se agruparán para traducir
        paquete_count = 0
        buffer_traslator = []
        buffer_cabeceras = []
        agrupando = False
        for line in infile:
            if primera_linea:
                outfile.write(line) #WEBVTT
                primera_linea = False
            es_cabecera_temporal = re.match(r'^(\d{2}:)?\d{2}:\d{2}\.\d{3} --> (\d{2}:)?\d{2}:\d{2}\.\d{3}', line)
            if es_cabecera_temporal and paquete_count == 0:
                agrupando = True
                buffer_cabeceras = []
                # outfile.write(line)
            if agrupando:
                if es_cabecera_temporal:
                    buffer_cabeceras.append(line)
                if not es_cabecera_temporal and line.strip() != "":
                    paquete_count += 1
                    buffer_traslator.append(line.strip() + f" {SEPARADOR} ")
                    if line.strip()[-1] == "." and paquete_count > 6 or paquete_count == max_paquetes:
                        #hemos terminado de agrupar ahora toca traducir y después
                        paquete_count = 0
                        agrupando = False
                        texto_a_traducir = "".join(buffer_traslator)
                        print("\ntexto_a_traducir:", texto_a_traducir)
                        translated_sentence = traducir(tipo_traductor, texto_a_traducir, origen, destino)
                        if texto_a_traducir.count(SEPARADOR) > translated_sentence.count(SEPARADOR):
                            translated_sentence = parchearSeparadoresPerdidos(texto_a_traducir, translated_sentence)
                        buffer_traslator = []
                        print("\ntranslated_sentence:", translated_sentence, "\n")
                        csv_buffer += texto_a_traducir + "~" + translated_sentence + "\n"

                        # quitar último $
                        ultimo_dolar_index = translated_sentence.rfind(SEPARADOR)
                        translated_sentence = translated_sentence[:ultimo_dolar_index]

                        # translated_sentence = translated_sentence.replace(" $", " ") 

                        # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
                        translated_sentence_list = translated_sentence.split(SEPARADOR)
                        print("translated_sentence_list:", translated_sentence_list)
                        for i in range(len(translated_sentence_list)):
                            indice = min(i, len(buffer_cabeceras)-1)
                            outfile.write(buffer_cabeceras[indice])
                            outfile.write(translated_sentence_list[indice].strip() + "\n")
                            outfile.write("\n")
                        


        if paquete_count != 0: #hemos terminado pero se han quedado lineas sin escribir
            print("ULTIMO PAQUETE!!!")
            print("\nbuffer_traslator:", buffer_traslator)
            translated_sentence = traducir(tipo_traductor, "".join(buffer_traslator), origen, destino)
            buffer_traslator = []
            print("translated_sentence:", translated_sentence, "\n")
            # quitar último $
            ultimo_dolar_index = translated_sentence.rfind(SEPARADOR)
            translated_sentence = translated_sentence[:ultimo_dolar_index]

            # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
            translated_sentence_list = translated_sentence.split(SEPARADOR)
            print("translated_sentence_list:", translated_sentence_list)
            for i in range(len(translated_sentence_list)):
                outfile.write(buffer_cabeceras[i])
                outfile.write(translated_sentence_list[i].strip() + "\n")   

    # guardar csv_buffer en fichero
    with open(path_csv, 'w', encoding='utf-8') as outfile:
        outfile.write(csv_buffer)  

In [37]:
prueba1="…the kind of high-level, top-tier API that you'll be using most of the time is not # as ready to go for beginners as you're used to from Fast.ai. And so that's actually, I # think, a good thing. You're up to Lesson 4. You know the basic idea now of how Gradient # Descent works, and how parameters are learned as part of a flexible function. I think you're # ready to try… # We got… # using a somewhat lower level library that does a little bit less for you. So it's going # to be, you know, a little bit more work. It's still, it's a very well-designed library and # it's still reasonably high level, but you're going to learn to go a little bit deeper. #"

prueba2="...el tipo de API de alto nivel que usarás la mayor parte del tiempo no está # tan lista para principiantes como la que estás acostumbrado de Fast.ai. Y creo que eso es bueno. Estás en la lección 4. Usted sabe la idea básica ahora de cómo Gradiente # Descenso obras, y cómo los parámetros se aprenden como parte de una función flexible. Creo que estás # listo para intentar... # Tenemos... # usando una biblioteca de nivel algo más bajo que hace un poco menos por ti. Así que va a ser # un poco más de trabajo. Sigue siendo, es una biblioteca muy bien diseñada y # sigue siendo razonablemente de alto nivel, pero vas a aprender a ir un poco más profundo. #"

parchearSeparadoresPerdidos(prueba1, prueba2)

'...el tipo de API de alto nivel que usarás la mayor parte del tiempo no está #tan lista para principiantes como la que estás acostumbrado de Fast.ai. Y creo que eso # es bueno. Estás en la lección 4. Usted sabe la idea básica ahora de cómo Gradiente# Descenso obras, y cómo los parámetros se aprenden como parte de una función flexible. Creo que estás # listo para intentar... # Tenemos... # usando una biblioteca de nivel algo más bajo que hace un poco menos por ti. Así que va a ser # un poco más de trabajo. Sigue siendo, es una biblioteca muy bien diseñada y # sigue siendo razonablemente de alto nivel, pero vas a aprender a ir un poco más profundo. #'

In [ ]:
# cargar fichero csv "csv_buffer" que tiene 2 columans separadas por el caracter "~"
# la primera columna es el texto en inglés y la segunda columna es el texto en español
# se usa para entrenar al modelo Mistral
import pandas as pd
df = pd.read_csv(path_csv, sep="~", header=None)
df.columns = ["eng", "spa"]
df.head()
# df.to_csv(path_csv, sep="~", header=None, index=False)
# meter las 10 primeras filas en una lista de diccionarios
lista_diccionarios = df.head(10).to_dict('records')
lista_diccionarios

In [16]:
print("Traducimos de", nombre_transcripcion, "a", nombre_transcripcion_trad)
# podemos elegir entre "seamless" y "helsinki" (helinski es más rápido y más preciso y funciona bien para frases largas)
traducir_subtitulos("deepl", nombre_transcripcion, nombre_transcripcion_trad, "eng", "spa")

Traducimos de /mnt/d/DeveloperIA/video-translator/audio_transcription/Lesson_4_Practical_Deep_Learning_for_Coders_2022.vtt a /mnt/d/DeveloperIA/video-translator/audio_transcription/Lesson_4_Practical_Deep_Learning_for_Coders_2022_traducidos.vtt

texto_a_traducir: Hi everybody and welcome to Practical Deep Learning for Coders Lesson 4, which I think # is the lesson that a lot of the regulars in the community have been most excited about # because it's where we're going to get some totally new material, totally new topic we've # never covered before. We're going to cover natural language processing, NLP, and you'll # find there is indeed a chapter about that in the book, but we're going to do it in a # totally different way to how it's done in the book. In the book we do NLP using the # Fast.ai library, using Recurrent Neural Networks, RNNs. Today we're going to do something else, # which is we're going to do transformers. And we're not even going to use the Fast.ai library # at all, in 

In [67]:
tunel.esta_activo_tunel_ssh()

True

In [ ]:
tunel.cerrar_tunel_ssh()

In [68]:
tunel.obtener_informacion_ip()

'Dirección IP: 74.208.163.90\nPaís: US\nRegión: New York\nCiudad: New York City'

Como el generador T2S en español necesita de frases de más de 12 palabras (y menos de 25) para que funciones aceptablemente tenemos que procesar el .VTT para que agrupe

In [26]:
def timestamp_to_seconds(timestamp):
    parts = timestamp.split(':')
    m, s_ms = parts[0], parts[1]
    s, ms = s_ms.split('.') if '.' in s_ms else (s_ms, '000')
    total_seconds = int(m) * 60 + int(s) + int(ms) / 1000
    return total_seconds

def seconds_to_timestamp(seconds):
    m, s = divmod(seconds, 60)
    s_int = int(s)
    ms = int((s - s_int) * 1000)
    return f"{int(m):02d}:{s_int:02d}.{ms:03d}"

# # Probando las funciones
# test_timestamp = "00:05.123"
# test_seconds = 123

# print(f"Conversión de timestamp a segundos: {timestamp_to_seconds(test_timestamp)} segundos")
# print(f"Conversión de segundos a timestamp: {seconds_to_timestamp(test_seconds)}")


def transform_vtt_file(input_file_path, output_file_path):
    try:
        # Leer el contenido del archivo VTT de entrada
        with open(input_file_path, 'r', encoding='utf-8') as file:
            vtt_content = file.read()

        # Aplicar la transformación
        lines = vtt_content.split("\n")
        output_lines = ["WEBVTT"]
        i = 1

        while i < len(lines):
            line = lines[i].strip()
            
            if "-->" in line:  # We have a timestamp line
                start_time = line.split("-->")[0].strip()
                i += 1
                current_text = lines[i].strip()
                i += 1
                
                while i < len(lines) and not "-->" in lines[i]:
                    current_text += " " + lines[i].strip()
                    i += 1
                
                # Check if the current text line has less than 13 words
                if len(current_text.split()) < 13 and i < len(lines) and "-->" in lines[i]:
                    end_time = lines[i].split("-->")[1].strip()
                    i += 1
                    next_text = lines[i].strip()
                    i += 1
                    while i < len(lines) and not "-->" in lines[i]:
                        next_text += " " + lines[i].strip()
                        i += 1
                    current_text += " " + next_text
                else:
                    end_time = line.split("-->")[1].strip()

                output_lines.append(f"{start_time} --> {end_time}")
                output_lines.append(current_text)
                output_lines.append("")  # Adding an empty line to separate subtitles
            else:
                i += 1
                
        transformed_vtt_content = "\n".join(output_lines)

        # Escribir el contenido transformado en el archivo VTT de salida
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.write(transformed_vtt_content)
        
        return "Transformación completada exitosamente."

    except Exception as e:
        return f"Ocurrió un error durante la transformación: {e}"



print("Adaptamos la transcripción para que sea más larga y se pueda generar su voz mejor")
transform_vtt_file(nombre_transcripcion_trad, nombre_transcripcion_adaptada)


Adaptamos la transcripción para que sea más larga y se pueda generar su voz mejor


'Transformación completada exitosamente.'

In [ ]:
# print("Traducimos de", nombre_transcripcion, "a", nombre_transcripcion_trad)
# # podemos elegir entre "seamless" y "helsinki" (helinski es más rápido y más preciso y funciona bien para frases largas)
# traducir_subtitulos("helsinki", nombre_transcripcion_adaptada, nombre_transcripcion_trad, "eng", "spa")

### Otro tipo Traducción más compleja (este tipo desajusta los tiempos)

primero agrupamos los archivos .VTT en lineas más largas que terminan en signo de puntuación para tener más contexto en la traducción

In [ ]:
#NOTA ESTÁ REPETIDA MÁS ARRIBA
def merge_vtt_lines(lines):
    merged_lines = ['WEBVTT\n\n']  # Initialize with the WEBVTT header and an extra new line
    buffer_time_start = None
    buffer_time_end = None
    buffer_text = None

    for line in lines:
        line = line.strip()

        # If the line is a time header
        if re.match(r'\d{2}:\d{2}.\d{3} --> \d{2}:\d{2}.\d{3}', line):
            time_start, time_end = line.split(' --> ')
            
            if buffer_text is not None:
                # Check if the text in the buffer ends with a punctuation mark
                if re.search(r'[.!?;,:]', buffer_text[-1]):
                    # Add the text in the buffer to the list with its corresponding time
                    merged_lines.append(f"{buffer_time_start} --> {buffer_time_end}\n{buffer_text}\n\n")
                    # Reset buffer_time and buffer_text for the next segment
                    buffer_time_start = time_start
                    buffer_time_end = time_end
                    buffer_text = None
                else:
                    # Update the end time of the buffer
                    buffer_time_end = time_end
            
            else:
                buffer_time_start = time_start
                buffer_time_end = time_end
            
            continue  # Skip to the next iteration

        # If the line is a transcription and is not a WEBVTT header
        elif line and not line.lower() == 'webvtt':
            if buffer_text is None:
                buffer_text = line
            else:
                # Add the current line to the text in the buffer
                buffer_text += " " + line

    # Add any remaining text in the buffer
    if buffer_text is not None:
        merged_lines.append(f"{buffer_time_start} --> {buffer_time_end}\n{buffer_text}\n")

    return merged_lines



def adapt_vtt_file(archivo_origen, archivo_adaptado):
    # Leer el archivo origen línea por línea y almacenarlo en una lista
    with open(archivo_origen, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Utilizar la función `merge_vtt_lines` para obtener las líneas adaptadas
    merged_lines = merge_vtt_lines(lines)

    # Escribir las líneas adaptadas en el archivo destino
    with open(archivo_adaptado, 'w', encoding='utf-8') as file:
        file.writelines(merged_lines)




# Uso de ejemplo
adapt_vtt_file(nombre_transcripcion, nombre_transcripcion_adaptada)


Traducimos de inglés a español:

In [ ]:
print("Traducimos de", nombre_transcripcion, "a", nombre_transcripcion_trad)
# podemos elegir entre "seamless" y "helsinki" (helinski es más rápido y más preciso y funciona bien para frases largas)
traducir_subtitulos("helsinki", nombre_transcripcion_adaptada, nombre_transcripcion_trad, "eng", "spa")

# Generación de Voz (traducida) sincronizada con el video a partir de un .vtt

Preparamos para el S2T, los .vtt con cadenas que son demasiado largas para el generador de voz las dividiremos.

In [27]:
import re
import math

def seconds_to_str(seconds):
    # Obtener minutos, segundos y milisegundos
    minutes = int(seconds // 60)
    remaining_seconds = int(seconds % 60)
    milliseconds = int((seconds * 1000) % 1000)
    
    # Convertir a cadena en el formato deseado
    formatted_time = f"{minutes:02d}:{remaining_seconds:02d}.{milliseconds:03d}"
    return formatted_time

def split_long_lines(input_filename, output_filename):
    # Leer el archivo VTT de entrada
    with open(input_filename, 'r', encoding='utf-8') as f:
        vtt_text = f.read()

    # Regex para encontrar y separar los componentes de cada entrada VTT
    pattern = re.compile(r'(\d{2}:\d{2}.\d{3} --> \d{2}:\d{2}.\d{3})\n(.+?)\n', re.DOTALL)
    entries = pattern.findall(vtt_text)

    # Preparar la lista para las nuevas entradas
    new_entries = []

    # Recorrer cada entrada
    for entry in entries:
        start_time, end_time = entry[0].split(' --> ')
        start_time = int(start_time.split(':')[0]) * 60 + int(start_time.split('.')[0].split(':')[1])  + float(start_time.split('.')[1])/1000
        end_time = int(end_time.split(':')[0]) * 60 + int(end_time.split('.')[0].split(':')[1])  + float(end_time.split('.')[1])/1000

        time_difference = end_time - start_time

        text = entry[1]
        words = text.split()

        if len(words) >= 25:
            # Buscar el signo de puntuación que divide la línea más equitativamente
            best_index = -1
            best_diff = float('inf')

            for i, word in enumerate(words):
                if word[-1] in ('.', ';', ',', ':'):
                    diff = abs(len(words[:i+1]) - len(words[i+1:]))
                    if diff < best_diff:
                        best_diff = diff
                        best_index = i

            first_half_words = words[:best_index+1]
            second_half_words = words[best_index+1:]

            # Calcular los nuevos tiempos
            total_words = len(first_half_words) + len(second_half_words)
            first_end_time = start_time + time_difference * (len(first_half_words) / total_words)
            first_end_time = round(first_end_time, 3)

            # Agregar las nuevas entradas
            new_entries.append((f"{seconds_to_str(start_time)} --> {seconds_to_str(first_end_time)}", ' '.join(first_half_words)))
            new_entries.append((f"{seconds_to_str(first_end_time)} --> {seconds_to_str(end_time)}", ' '.join(second_half_words)))
        else:
            new_entries.append(entry)

    # Escribir las nuevas entradas al archivo VTT de salida
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write("WEBVTT\n\n")
        for entry in new_entries:
            f.write(f"{entry[0]}\n{entry[1]}\n\n")


# simplifico lineas largas
split_long_lines(nombre_transcripcion_adaptada, nombre_transcripcion_lista_t2s)
# lo hacemos por 2 veces para que no quede ninguna linea con mas de 25 palabras
# split_long_lines(nombre_transcripcion_lista_t2s, nombre_transcripcion_lista_t2s)


In [28]:
split_long_lines(nombre_transcripcion_lista_t2s, nombre_transcripcion_lista_t2s)

## Preprocesamiento antes del T2S para ayudar a las limitaciones del modelo T2S

In [18]:
from pydub import AudioSegment, silence

def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
    """
    Elimina silencios largos de un archivo de audio.

    Parámetros:
    - input_filepath: ruta al archivo de audio de entrada (MP3).
    - output_filepath: ruta al archivo de audio de salida (MP3).
    - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
    - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
    - silence_thresh: umbral de silencio (en dB).
    """
    
    # Cargar el archivo de audio
    audio_segment = AudioSegment.from_wav(input_filepath)
    
    # Encuentra los segmentos de audio separados por silencios
    segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

    # Crear un nuevo segmento de audio con silencios ajustados
    new_audio_segment = AudioSegment.empty()
    silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

    # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
    for segment in segments:
        new_audio_segment += segment + silence_chunk

    # Removemos el último chunk de silencio añadido
    new_audio_segment = new_audio_segment[:-new_silence_len]

    # Guarda el nuevo archivo de audio
    new_audio_segment.export(output_filepath, format="wav")


Para evaluar las generaciones de voz podemos usar whisper para volver a hacer la transcripción de la voz generada y ver si coincide con la original

In [14]:
import whisper
modelWhisper = whisper.load_model('small')

In [7]:
import Levenshtein

texto1= "hola como estas"
texto2= "hola, como estas?"
# funcion que quita simbolos de puntuacion
def quitar_puntuacion(texto):
    texto = texto.replace('.', '')
    texto = texto.replace(',', '')
    texto = texto.replace('?', '')
    texto = texto.replace('!', '')
    texto = texto.replace('¿', '')
    texto = texto.replace('¡', '')
    texto = texto.replace(';', '')
    texto = texto.replace(':', '')
    texto = texto.replace('(', '')
    texto = texto.replace(')', '')
    texto = texto.replace('[', '')
    texto = texto.replace(']', '')
    texto = texto.replace('{', '')
    texto = texto.replace('}', '')
    texto = texto.replace('"', '')
    texto = texto.replace("'", '')
    texto = texto.replace('`', '')
    texto = texto.replace('´', '')
    texto = texto.replace('’', '')
    texto = texto.replace('‘', '')
    texto = texto.replace('“', '')
    texto = texto.replace('”', '')
    texto = texto.replace('…', '')
    texto = texto.replace('«', '')
    texto = texto.replace('»', '')
    texto = texto.replace('–', '')
    texto = texto.replace('—', '')
    texto = texto.replace('−', '')
    texto = texto.replace('‐', '')
    texto = texto.replace('⁃', '')
    texto = texto.replace('‒', '')
    texto = texto.replace('―', '')
    texto = texto.replace('‹', '')
    texto = texto.replace('›', '')
    texto = texto.replace('•', '')
    texto = texto.replace('·', '')
    texto = texto.replace('‣', '')
    texto = texto.replace('⁌', '')
    texto = texto
    return texto

# funcion que calcula la distancia entre dos textos
Levenshtein.ratio(quitar_puntuacion(texto1), quitar_puntuacion(texto2))

1.0

El modelo no entiende de números aritméticos

In [8]:
import re
from num2words import num2words

def number_to_words(num_str):
    try:
        num = int(num_str)
        return num2words(num, lang='es')
    except ValueError:
        return "Por favor, introduzca un número válido."

def process_numbers_in_line(line):
    def replace_with_words(match):
        return number_to_words(match.group())
    
    return re.sub(r'\b\d+\b', replace_with_words, line)

# Ejemplo de uso
line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
new_line = process_numbers_in_line(line)
print(new_line)  
# Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."

Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas.


In [9]:
# Diccionario con las traducciones


def process_abrev(line):
    translations = {
    'Dr': 'doctor',
    'Sr': 'señor',
    'Sra': 'señora',
    # Añade más traducciones aquí
}
    for abbr, full in translations.items():
        line = line.replace(f'{abbr}.', full)
        line = line.replace(f'{abbr} ', f'{full} ')
    return line

# Ejemplo de uso
line1 = 'El Dr. está hablando con el Sr. Pérez y la Sra. Gómez.'
line2 = 'Buenos días Sr. Gómez.'

new_line1 = process_abrev(line1)
new_line2 = process_abrev(line2)

print(new_line1)  # Debería imprimir: 'El Doctor está hablando con el Señor Pérez y la Señora Gómez.'
print(new_line2)  # Debería imprimir: 'Buenos días Señor Gómez.'

El doctor está hablando con el señor Pérez y la señora Gómez.
Buenos días señor Gómez.


In [10]:
# Diccionario con las traducciones


def otras_traducciones(line):

    translations = {
    '-': ',',
    '—': ',', 
    '%': ' por ciento '
    # Añade más traducciones aquí
    }
    
    for old, new in translations.items():
        line = line.replace(old, new)
    return line

# Ejemplo de uso
line1 = 'Hola —cómo estás—?, bien al 100%, gracias.'
line2 = "El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón —el primer hobbit que fue famoso en el mundo entero— y que él tituló Historia de una ida y de una vuelta,"

new_line1 = otras_traducciones(line1)
new_line2 = otras_traducciones(line2)

print(new_line1)  # Debería imprimir: 'Hola, cómo estás?'
print(new_line2)  # Debería imprimir: 'Bien, gracias.'

Hola ,cómo estás,?, bien al 100 por ciento , gracias.
El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón ,el primer hobbit que fue famoso en el mundo entero, y que él tituló Historia de una ida y de una vuelta,


In [11]:
def preprocesado_al_modelo(line):
    line_with_numbers = process_numbers_in_line(line)
    line_with_both = process_abrev(line_with_numbers)
    line_with_all = otras_traducciones(line_with_both)
    return line_with_all

## modelo generador de voz

In [12]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
print(os.environ['TF_ENABLE_ONEDNN_OPTS'])  # Salida: mi_valor

0


Cargamos el modelo generador fairseq

In [4]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd
import torch  # Importamos PyTorch para poder usar la función `to()`

# Función para mover recursivamente todos los tensores en una estructura anidada a un dispositivo
def move_to_device(obj, device):
    if isinstance(obj, torch.Tensor):
        return obj.to(device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, device) for item in obj]
    else:
        return obj
    

# Cargamos el modelo y la configuración desde el modelo preentrenado de Hugging Face
models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-es-css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]

# Movemos el modelo al dispositivo GPU
model = model.to('cuda:0')

# Actualizamos la configuración con los datos del task
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)

# Creamos el generador
generator = task.build_generator([model], cfg)


# Texto a convertir en voz
text = "Hola, esta es una prueba. Quiero saber hasta dónde llega el modelo."

# Preparamos los datos de entrada para el modelo
sample = TTSHubInterface.get_model_input(task, text)

# Movemos los datos al dispositivo GPU
sample = move_to_device(sample, 'cuda:0')

# Realizamos la predicción
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

# Convertimos el resultado al formato de audio y lo reproducimos
ipd.Audio(wav.cpu(), rate=rate)  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio


2023-11-02 22:23:39 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

2023-11-02 22:23:41 | INFO | fairseq.tasks.speech_to_text | dictionary size (spm_char.txt): 107
/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
2023-11-02 22:23:41 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin
2023-11-02 22:23:42 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/javier/.cache/fairseq/models--facebook--tts_transformer-es-css10/snapshots/f52cf36f741df546bed60cdd5e6b71e0b85378c1/hifigan.bin
/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning:

Definimos interfaz que vamos a usar para usar el modelo y generar voz

In [5]:
def spanish_t2s(text):
    text = preprocesado_al_modelo(text)
    # Preparamos los datos de entrada para el modelo
    sample = TTSHubInterface.get_model_input(task, text)

    # Movemos los datos al dispositivo GPU
    sample = move_to_device(sample, 'cuda:0')

    # Realizamos la predicción
    wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

    return wav, rate

In [15]:
import torchaudio
import IPython.display as ipd


texto_prueba = preprocesado_al_modelo("Hola Sr. usuario, esta es una prueba de 2 pasos. Quiero saber hasta dónde llega el modelo.")
print("texto_prueba:", texto_prueba)

wav, rate = spanish_t2s(texto_prueba)

if len(wav.shape) == 1:
    wav = wav.unsqueeze(0)

temp_file_name = "ejemploTemporal.wav"
torchaudio.save(temp_file_name, wav.to('cpu'), rate)


prediction = modelWhisper.transcribe("ejemploTemporal.wav", language="Spanish", temperature=0, beam_size=1)["text"]
print("prediction:", prediction)
similitud = Levenshtein.ratio(quitar_puntuacion(texto_prueba), quitar_puntuacion(prediction))

print("similitud:", similitud)
# escuchar audio
ipd.Audio(wav.cpu(), rate=rate)  # Usamos `.cpu()` para mover los datos de vuelta a la CPU antes de reproducir el audio

texto_prueba: Hola señor usuario, esta es una prueba de dos pasos. Quiero saber hasta dónde llega el modelo.
prediction:  Hola señor usuario, esta es una prueba de dos pasos. Quiero saber hasta dónde llega el modelo.
similitud: 0.994535519125683


De un fichero .vtt crea una lista de pares (tiempo, texto)

In [16]:
import re

def vtt_to_couple_time_text_list(filename):
    # Leemos el contenido del archivo
    with open(filename, 'r', encoding='utf-8') as f:
        vtt_content = f.read()

    # Usamos una expresión regular para identificar y eliminar las líneas de tiempo
    pattern = re.compile(r'^\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}\.\d{3}$')
    
    # Dividimos el contenido por líneas
    lines = vtt_content.split('\n')

    # Filtramos para quedarnos solo con las líneas de tiempo
    time_lines = [line for line in lines if pattern.match(line)]

    # Filtramos las líneas que no queremos: líneas de tiempo y líneas vacías
    text_lines = [line for line in lines if not pattern.match(line) and line.strip() != "" and line != "WEBVTT"]

    # creamos una nueva lista de pares (tiempo, texto)
    time_text_pairs = list(zip(time_lines, text_lines))

    return time_text_pairs

In [ ]:
# import os
# import torchaudio
# import Levenshtein
# import shutil


# small_temp_files = []

# # crear carpeta temporal si no existe
# carpetaTemporal = "./asudioLibro_sp"
# if not os.path.exists(carpetaTemporal):
#     os.makedirs(carpetaTemporal)

# # Itera sobre cada párrafo
# for i, text in enumerate(parrafos):
#     print(f"Paragraph {i + 1}:\n{text}\n")


#     text = parrafos[i]
#     text = preprocesado_al_modelo(text)

#     # text = "pues, contaba el viaje de Bilbo hacia el este y la vuelta,"
#     print(f"Paragraph {i + 1}:\n{text}\n")
#     # Genera el audio

#     wav, rate = spanish_t2s(text)

#     if len(wav.shape) == 1:
#         wav = wav.unsqueeze(0)

#     temp_file_name = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.wav"
#     torchaudio.save(temp_file_name, wav.to('cpu'), rate)

#     similitud = 0
#     contador = 0
#     line = text
#     longitud_texto = len(text.split())
#     while True:
        
#         prediction = modelWhisper.transcribe(temp_file_name, language="Spanish", temperature=0, beam_size=1)["text"]
#         similitud = Levenshtein.ratio(quitar_puntuacion(line), quitar_puntuacion(prediction))
#         print("Linea: ", line)
#         print("Prediction: ", prediction)
#         print("Similitud: ", similitud)
#         print("Contador: ", contador)
        

#         print("Provisional transcripted_sentence:", prediction)
#         if (similitud > 0.96) or (similitud > 0.93 and longitud_texto < 24) or (similitud > 0.90 and contador > 8) or (similitud > 0.88 and contador > 11) or (similitud > 0.85 and contador > 15) or contador > 17:
#             break
#         else:
#             contador += 1
#             if contador > 2 and longitud_texto < 24 or contador > 4:
#                 line = " ".join(text.split()[:contador]) + "," + " ".join(text.split()[contador:])
#             wav, rate = spanish_t2s(line)
#             if len(wav.shape) == 1:
#                 wav = wav.unsqueeze(0)
#             torchaudio.save(temp_file_name, wav.to('cpu'), rate)

#     temp_file_name_definitivo = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.mp3"
#     # convertir .wav en .mp3
#     !ffmpeg -y -i "{temp_file_name}" "{temp_file_name_definitivo}"
#     # eliminar .wav
#     !rm "{temp_file_name}"
#     # añadir fichero a la lista de ficheros temporales
#     small_temp_files.append(temp_file_name_definitivo)


# # Concatena todos los archivos mp3 grandes en uno final
# !ffmpeg -y -f concat -safe 0 -i <(for f in {' '.join(small_temp_files)}; do echo file ${"PWD"}/${"f"}; done) -c copy "{carpetaTemporal}/audioLibro_output.mp3"

# # quitamos silencios
# quitar_silencios(f"{carpetaTemporal}/audioLibro_output.mp3", f"{carpetaTemporal}/audioLibro_output_sin_silencios.mp3")
# !cp "{carpetaTemporal}/audioLibro_output_sin_silencios.mp3" "./audioLibro_sp_output_sin_silencios.mp3"
# shutil.rmtree(carpetaTemporal)

A partir de un fichero .vtt ya traducido te crea el audio sincronizado

In [19]:
from pydub import AudioSegment
import torchaudio
import os
from datetime import datetime, timedelta


def time_to_milliseconds(time_str):
    time_obj = datetime.strptime(time_str, '%M:%S.%f')
    # time_obj = datetime.strptime(time_str, '%H:%M:%S.%f')
    # return int(timedelta(hours=time_obj.hour, minutes=time_obj.minute, seconds=time_obj.second, microseconds=time_obj.microsecond).total_seconds() * 1000)
    return int(timedelta(minutes=time_obj.minute, seconds=time_obj.second).total_seconds() * 1000)

carpetaTemporal = "./aProfesionalTraductor_sp"

# crear carpeta temporal si no existe
if not os.path.exists(carpetaTemporal):
    os.makedirs(carpetaTemporal)

small_temp_files = []

current_time_ms = 0

paragraphs = vtt_to_couple_time_text_list(nombre_transcripcion_lista_t2s)

delta = 0
silencio = 0
acumulado = 0
umbral = 100 #milisegundos
playback_speed_limit = 1.1
silence_index = 0.5

# Itera sobre cada par (tiempo, texto) en la lista
for i, (time_range, text) in enumerate(paragraphs):
    start_time, end_time = time_range.split(' --> ')
    start_time_ms = time_to_milliseconds(start_time)
    end_time_ms = time_to_milliseconds(end_time)

    # Genera el audio
    # !python3 tortoise/do_tts.py --voice=$"{voice}" --candidates=1 --text=$"{text}" --preset=$"{preset}" --output_path=./$"{voice}"/
    wav, rate = spanish_t2s(text)

    if len(wav.shape) == 1:
        wav = wav.unsqueeze(0)

    temp_file_name = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.wav"
    torchaudio.save(temp_file_name, wav.to('cpu'), rate)

    # quitamos silencios
    quitar_silencios(temp_file_name, temp_file_name, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)

    # obtiene audio_segment a partir de un archivo wav
    audio_segment = AudioSegment.from_wav(temp_file_name)

    # frecuencia de muestreo
    print("sample_rate:", audio_segment.frame_rate)

    # eliminar archivo mp3
    # !rm ./{voice}/{voice}_0.wav

    # gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)
    # audio_segment = gen.squeeze(0).cpu()

    # En pydub la función len() devuelve la duración en milisegundos
    audio_duration_ms = len(audio_segment)

    print("segmento:", i)
    # Calcula la diferencia de tiempo
    # delta = end_time_ms - (start_time_ms + audio_duration_ms)
    # print("delta:", delta)
    # delta += acumulado
    # print("delta acumulado:", delta)

    print("start_time_ms", start_time_ms)
    print("end_time_ms", end_time_ms)
    video_duration_ms = end_time_ms - start_time_ms
    print("video_duration_ms", video_duration_ms)
    print("audio_duration_ms", audio_duration_ms)

    delta = video_duration_ms - audio_duration_ms
    print("delta:", delta)
    acumulado += delta
    print("acumulado:", acumulado)

    # En el caso especial de la primera iteración hay que añadir un silencio al principio
    if i == 0 and delta > 0:
        silence_segment = AudioSegment.silent(duration = delta * 0.9)
        audio_segment = silence_segment + audio_segment
        # desacumulamos el tiempo silenciado
        acumulado = delta * (1 - 0.9)
        print("silencio: ", delta * 0.9)
    else:

        # Ajusta la velocidad si es necesario (dejamos un umbral para agilizar el proceso)
        if acumulado < -umbral:
            multiplicador = 1 +  (abs(acumulado) / audio_duration_ms)
            if multiplicador > playback_speed_limit:
                playback_speed = playback_speed_limit
                # acumulado = -(multiplicador - playback_speed_limit)*audio_duration_ms
            else:
                playback_speed = multiplicador
            #acumulamos el tiempo que nos falta para llegar al tiempo del video
            acumulado += (multiplicador - playback_speed) * audio_duration_ms
            audio_segment = audio_segment.speedup(playback_speed = playback_speed)
            print("acelerado: x", playback_speed)
        elif acumulado > umbral:
            if acumulado < 1000:
                silence_index = 0.5
            elif acumulado < 2000:
                silence_index = 0.65
            elif acumulado < 3000:
                silence_index = 0.75
            else:
                silence_index = 0.90
            silencio = acumulado * silence_index
            # silence_segment = AudioSegment.silent(duration = silencio)
            # si el texto termina en "." añadimos todo el silencio extra al final del segmento
            if text[-1] == ".":
            # añadimos solo la mitad de delta para intentar luego compensar cuando se acelera
                audio_segment =  audio_segment + AudioSegment.silent(duration = silencio) 
            else:
                #lo añadimos la mitad al principio y la otra mitad al final
                audio_segment = AudioSegment.silent(duration = silencio / 2) + audio_segment + AudioSegment.silent(duration = silencio / 2)
            # desacumulamos el tiempo silenciado
            acumulado -= silencio
            # audio_segment += silence_segment
            print("silencio: ", silencio)
    
    print("acumuladoFinal:", acumulado)
        
 
    small_temp_file_name = f'./{carpetaTemporal}/small_temp_audio_{str(i).zfill(5)}.mp3'
    # Exportar como MP3
    audio_segment.export(small_temp_file_name, format="mp3")
    # torchaudio.save(temp_file_name, audio_segment, 24000, format="mp3")
    small_temp_files.append(small_temp_file_name)

    # Eliminar archivo .wav
    os.remove(temp_file_name)


# Concatena todos los archivos mp3 grandes en uno final
!ffmpeg -y -f concat -safe 0 -i <(for f in {' '.join(small_temp_files)}; do echo file ${"PWD"}/${"f"}; done) -c copy "./{carpetaTemporal}/ProfesionalTraductor_output.mp3"

!cp "./{carpetaTemporal}/ProfesionalTraductor_output.mp3" ../



# Elimina archivos mp3 grandes
os.system(f"rm {' '.join(small_temp_files)}")


sample_rate: 22050
segmento: 0
start_time_ms 0
end_time_ms 10000
video_duration_ms 10000
audio_duration_ms 7245
delta: 2755
acumulado: 2755
silencio:  2479.5
acumuladoFinal: 275.49999999999994
sample_rate: 22050
segmento: 1
start_time_ms 12000
end_time_ms 21000
video_duration_ms 9000
audio_duration_ms 8081
delta: 919
acumulado: 1194.5
silencio:  776.4250000000001
acumuladoFinal: 418.07499999999993
sample_rate: 22050
segmento: 2
start_time_ms 21000
end_time_ms 26000
video_duration_ms 5000
audio_duration_ms 4070
delta: 930
acumulado: 1348.0749999999998
silencio:  876.2487499999999
acumuladoFinal: 471.82624999999996
sample_rate: 22050
segmento: 3
start_time_ms 26000
end_time_ms 32000
video_duration_ms 6000
audio_duration_ms 6316
delta: -316
acumulado: 155.82624999999996
silencio:  77.91312499999998
acumuladoFinal: 77.91312499999998
sample_rate: 22050
segmento: 4
start_time_ms 32000
end_time_ms 38000
video_duration_ms 6000
audio_duration_ms 6641
delta: -641
acumulado: -563.086875
acelerado

0

Baja el volumen del audio original

In [20]:
from pydub import AudioSegment

# Cargar el segmento de audio desde un archivo mp3
audio_segment = AudioSegment.from_mp3(output_audio)

# Bajar el volumen en 10 dB
audio_segment_quiet = audio_segment - 23

# Guardar el nuevo segmento de audio a un archivo
audio_segment_quiet.export(f"{path_video}_volumen_disminuido.mp3", format="mp3")

<_io.BufferedRandom name='/mnt/d/DeveloperIA/video-translator/media/Lectura22.mp4_volumen_disminuido.mp3'>

Creamos video con voz en off traduciendo

In [22]:
!ffmpeg -y -i "{path_video}" -i "{path_video}_volumen_disminuido.mp3" -i "{carpetaTemporal}/ProfesionalTraductor_output.mp3" \
-filter_complex "[1:a][2:a]amix=inputs=2:duration=first[aout]" \
-map 0:v -map "[aout]" -c:v copy -c:a aac {path_video}_con_audio_mezclado.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

## Bajar dataset de traducciones de opus-100-corpus

In [ ]:
!wget https://object.pouta.csc.fi/OPUS-100/v1.0/opus-100-corpus-en-es-v1.0.tar.gz
!mv opus-100-corpus-en-es-v1.0.tar.gz datasets/
# descomprimir archivo ./datasets/opus-100-corpus-en-es-v1.0.tar.gz en ./datasets
!tar -xvzf ./datasets/opus-100-corpus-en-es-v1.0.tar.gz -C ./datasets

## traducciones paralelas de novelas e historias (es un buen dataset para entrenar modelos de traducción)
https://albalearning.com/audiolibros/textosparalelos.html

https://chat.openai.com/share/beb3bd9d-7375-4f9a-a505-36fb3a464192


Pasamos el dataset de formato .txt donde cada linea es una frase en el idioma original y su traducción en el idioma destino a formato .csv para que lo entienda el modelo de traducción

In [9]:
# import csv

# def text_to_csv(input_file, output_file):
#     # Extract the root name of the input file
#     # file_name = input_file.split('.')[0]

#     # Read the text file
#     with open(input_file, 'r') as file:
#         text_data = file.read()

#     # Split the text into lines
#     lines = text_data.strip().split('\n')

#     # Prepare the CSV file
#     # output_file = f'{file_name}.csv'
#     with open(output_file, 'w', newline='') as file:
#         csv_writer = csv.writer(file, delimiter='~')
#         csv_writer.writerow(['spanish', 'english'])  # Write headers

#         # Process the lines assuming each Spanish line is followed by its English translation
#         for i in range(0, len(lines), 2):
#             spanish_text = lines[i].replace("--", ",")
#             english_text = lines[i + 1].replace("--", ",") #if i + 1 < len(lines) else ""
#             csv_writer.writerow([spanish_text, english_text])

#     return output_file

# input_file = 'datasets/novelas/al_otro_lado_de_la_pared.txt'
# output_file = text_to_csv(input_file, 'datasets/novelas/al_otro_lado_de_la_pared.csv')
# print(f'CSV file created: {output_file}')


CSV file created: datasets/novelas/al_otro_lado_de_la_pared.csv


In [9]:
import csv

def text_to_csv(input_file, output_file):
    # Extract the root name of the input file
    # file_name = input_file.split('.')[0]

    # Read the text file
    with open(input_file, 'r') as file:
        text_data = file.read()

    # Split the text into lines
    lines = text_data.strip().split('\n')

    # Prepare the CSV file
    # output_file = f'{file_name}.csv'
    with open(output_file, 'w', newline='') as file:
        
        file.write("spanish~english\n")

        # Process the lines assuming each Spanish line is followed by its English translation
        for i in range(0, len(lines), 2):
            spanish_text = lines[i].replace("--", ",")
            english_text = lines[i + 1].replace("--", ",") #if i + 1 < len(lines) else ""
            file.write(f"{spanish_text}~{english_text}\n")
    return output_file

input_file = 'datasets/novelas/al_otro_lado_de_la_pared.txt'
output_file = text_to_csv(input_file, 'datasets/novelas/al_otro_lado_de_la_pared.csv')
print(f'CSV file created: {output_file}')


CSV file created: datasets/novelas/al_otro_lado_de_la_pared.csv


In [90]:
import pandas as pd
df = pd.read_csv(r'./datasets/novelas/csv/el_amigo_fiel.csv', sep='~', quoting=csv.QUOTE_NONE)
df.head()

,spanish,english
0,"Una mañana, la vieja rata de agua sacó la cabe...",One morning the old Water-rat put his head out...
1,-No podréis ir nunca a la buena sociedad si n...,"""You will never be in the best society unless ..."
2,-¡Qué criaturas más desobedientes! -exclamó ...,"""What disobedient children!"" cried the old Wat..."
3,-¡No lo quiera Dios! -replicó la pata-. Todo t...,"""Nothing of the kind,"" answered the Duck, ""eve..."
4,-¡Ah! No tengo la menor idea de los sentimient...,"""Ah! I know nothing about the feelings of pare..."


#### Creamos script que procese el dataset de traducciones paralelas y lo convierta a formato .csv

In [10]:
import os

novelas_dir = "datasets/novelas/"
output_dir = "datasets/novelas/csv/"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate through all files in the directory
for filename in os.listdir(novelas_dir):
    if filename.endswith(".txt"):
        input_file = os.path.join(novelas_dir, filename)
        output_file = os.path.join(output_dir, f"{filename.split('.')[0]}.csv")
        output_file = text_to_csv(input_file, output_file)
        print(f'CSV file created: {output_file}')


CSV file created: datasets/novelas/csv/abuelita.csv
CSV file created: datasets/novelas/csv/al_otro_lado_de_la_pared.csv
CSV file created: datasets/novelas/csv/caperucita_roja.csv
CSV file created: datasets/novelas/csv/dagon.csv
CSV file created: datasets/novelas/csv/de_la_oscuridad.csv
CSV file created: datasets/novelas/csv/diagnostico_de_muerte.csv
CSV file created: datasets/novelas/csv/elefantes_enamorados.csv
CSV file created: datasets/novelas/csv/el_amigo_fiel.csv
CSV file created: datasets/novelas/csv/El_arbol.csv
CSV file created: datasets/novelas/csv/el_arte_de_leer.csv
CSV file created: datasets/novelas/csv/el_ceremonial.csv
CSV file created: datasets/novelas/csv/el_clerigo_malvado.csv
CSV file created: datasets/novelas/csv/el_corazon_delator.csv
CSV file created: datasets/novelas/csv/el_crimen_de_la_calle_la_perseguida.csv
CSV file created: datasets/novelas/csv/el_entierro_prematuro.csv
CSV file created: datasets/novelas/csv/el_gigante_egoista.csv
CSV file created: datasets/no

CSV file created: datasets/novelas/csv/el_jardinero.csv
CSV file created: datasets/novelas/csv/el_lobo_y_el_cabritillo.csv
CSV file created: datasets/novelas/csv/el_pozo_y_el_pendulo.csv
CSV file created: datasets/novelas/csv/el_principe_feliz.csv
CSV file created: datasets/novelas/csv/el_retrato_oval.csv
CSV file created: datasets/novelas/csv/el_ruiseñor_y_la_rosa.csv
CSV file created: datasets/novelas/csv/el_silencio_de_las_sirenas.csv
CSV file created: datasets/novelas/csv/el_sueño.csv
CSV file created: datasets/novelas/csv/Encender_una_hoguera.csv
CSV file created: datasets/novelas/csv/En_el_camino_de_Brighton.csv
CSV file created: datasets/novelas/csv/en_el_valle_de_la_sombra.csv
CSV file created: datasets/novelas/csv/en_la_cripta.csv
CSV file created: datasets/novelas/csv/ex_oblivione.csv
CSV file created: datasets/novelas/csv/laura.csv
CSV file created: datasets/novelas/csv/La_bestia_en_la_cueva.csv
CSV file created: datasets/novelas/csv/la_calle.csv
CSV file created: datasets/n

Verificamos que todas las lineas tienen el mismo número de columnas (solo tienen 1 caracter ~ que es el separador de columnas)

In [47]:
def verificar_lineas_con_caracter(fichero):
    """
    Verifica que cada línea del fichero contenga exactamente un caracter '~'.
    Devuelve una lista de pares (número de línea, string de la línea) para las
    líneas que no cumplen con esta condición.

    :param fichero: Ruta al archivo de texto a verificar.
    :return: Lista de pares (número de línea, string de la línea).
    """
    lineas_incorrectas = []

    with open(fichero, 'r') as archivo:
        for numero_linea, linea in enumerate(archivo, start=1):
            if linea.count('~') != 1:
                lineas_incorrectas.append((numero_linea, linea.strip()))

    return lineas_incorrectas

# Uso de ejemplo:
# Suponiendo que 'ruta_al_archivo.txt' es la ruta al archivo de texto
# lineas_incorrectas = verificar_lineas_con_caracter('ruta_al_archivo.txt')
# print(lineas_incorrectas)

# verificar todas los archivos csv de la carpeta ./datasets/novelas/csv
import os
import glob

# Carpeta con los archivos CSV
csv_folder = './datasets/novelas/csv'

# Obtener la lista de archivos CSV
csv_files = glob.glob(os.path.join(csv_folder, '*.csv'))

# Verificar cada archivo CSV
for csv_file in csv_files:
    lineas_incorrectas = verificar_lineas_con_caracter(csv_file)
    if lineas_incorrectas:
        print(f'Archivo CSV con líneas incorrectas: {csv_file}')
        print('Líneas incorrectas:')
        for numero_linea, linea in lineas_incorrectas:
            print(f'  Línea {numero_linea}: {linea}')

comprobamos que todos los archivos csv se pueden abrir con el parseador de pandas

In [89]:
import os
import glob

# Carpeta con los archivos CSV
csv_folder = './datasets/novelas/csv'

# Obtener la lista de archivos CSV
csv_files = glob.glob(os.path.join(csv_folder, '*.csv'))

# Verificar cada archivo CSV puede ser leído por pandas
for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, sep='~', quoting=csv.QUOTE_NONE)
        # print(f'Archivo CSV válido: {csv_file}')
        # print(df.head())
    except Exception as e:
        print(f'Error al leer el archivo CSV: {csv_file}')
        print(f'Error: {e}')

Agrupamos todos los csv en uno solo

In [68]:
%%bash
# Crear un nuevo archivo con la cabecera deseada
echo "Spanish~English" > opus_books_dataset.csv

# Concatenar todos los archivos omitiendo su primera línea
for file in datasets/novelas/csv/*.csv; do
    tail -n +2 "$file" >> novelas_cortas_sp_en.csv
done

In [92]:
verificar_lineas_con_caracter('novelas_cortas_sp_en.csv')

[]

In [80]:
!find ./datasets/novelas/csv/ -type f -name '*.csv' -exec grep 'Todo el mundo asistió al entierro del pequeño Hans' {} +

./datasets/novelas/csv/el_amigo_fiel.csv:Todo el mundo asistió al entierro del pequeño Hans porque era muy querido. Y el molinero figuró a la cabeza del duelo.~"Everybody went to little Hans' funeral, as he was so popular, and the Miller was the chief mourner.


In [91]:
import pandas as pd
df = pd.read_csv(r'./novelas_cortas_sp_en.csv', sep='~', quoting=csv.QUOTE_NONE) #IMPORTANTE poner quoting=csv.QUOTE_NONE para que no se queje de las comillas
df.head()

,Spanish,English
0,"En una verde ladera del monte Menalo, en Arcad...","On a verdant slope of Mount Maenalus, in Arcad..."
1,"Hace muchos años, cuando la villa de la cuesta...","Many years ago, when the hillside villa was ne..."
2,Pero aunque Calos y Musides estaban en perfect...,But though Kalos and Musides dwelt in unbroken...
3,Tan famosos eran Calos y Musides que a nadie l...,"So famous were Kalos and Musides, that none wo..."
4,Los escultores aceptaron complacidos el encarg...,With joy the sculptors hailed the Tyrant's off...


(opcional) Luego sería bueno agrupar parrafos hasta 4500 letras (deepl soporta hasta 5000) para que el modelo de traducción funcione mejor y optimizar el entrenamiento.

In [ ]:
import string

# Función para contar las letras en un texto, excluyendo espacios y signos de puntuación
def count_letters(text):
    return sum(1 for char in text if char.isalpha())

# Función para contar todas las letras de un texto (incluidas espacios y signos de puntuación)
def count_all_letters(text):
    return sum(1 for char in text if char in string.printable)



## traducciones paralelas de webs (NOTA: no son traducciones limpias muchas de ellas)

Parseo fichero descargado de https://elrc-share.eu/repository/browse/spanish-english-website-parallel-corpus/0cb6cd0d13f811e7bfe700155d0205023170d9220cae4b2b8331fb9bdc7a4fa3/


In [8]:
from xml.etree import ElementTree as ET
import csv
import io

# Simulando el archivo XML basado en el ejemplo proporcionado
xml_data = """
<tmx>
  <body>
    <tu tuid="1">
      <tuv xml:lang="en">
        <seg>Home Help Computer enquiries Categories Digital signature (certificate or electronic ID card) and Cl@ve PIN system Electronic certificates Cómo realizar una copia de seguridad de su certificado electrónico Instrucciones para Macintosh Copia del certificado electrónico en Google Chrome (Mac)</seg>
      </tuv>
      <tuv xml:lang="es">
        <seg>Home Help Computer enquiries Categories Digital signature (certificate or electronic ID card) and Cl@ve PIN system Electronic certificates Cómo realizar una copia de seguridad de su certificado electrónico Instrucciones para Macintosh Copia del certificado electrónico en Safari (Mac)</seg>
      </tuv>
    </tu>
    <tu tuid="2">
      <tuv xml:lang="en">
        <seg>Copia del certificado electrónico en Google Chrome (Mac)</seg>
      </tuv>
      <tuv xml:lang="es">
        <seg>Copia del certificado electrónico en Safari (Mac)</seg>
      </tuv>
    </tu>
    <tu tuid="4">
      <tuv xml:lang="en">
        <seg>The online Securities Portfolio service allows you to obtain an informative summary of operations carried out in 2015 relating to shares quoted on official Spanish secondary securities markets (Stock Exchanges) or on those defined in Directive 2004/39/EC of the European Parliament and the Council of 21 April 2004.</seg>
      </tuv>
      <tuv xml:lang="es">
        <seg>El programa on line de &quot;Servicio de Cartera de valores&quot; permite obtener un resumen informativo de las operaciones efectuadas en 2015 relativas a acciones con cotización en mercados secundarios oficiales de valores españoles (Bolsas de Valores) o en los definidos en la Directiva 2004/39/CE, del Parlamento Europeo y del Consejo, de 21 de abril de 2004.</seg>
      </tuv>
    </tu>
    <tu tuid="22959">
      <tuv xml:lang="en">
        <seg>Our doors are open and I would encourage you to view this site as a meeting point and a tool for promoting new projects and initiatives.</seg>
      </tuv>
      <tuv xml:lang="es">
        <seg>Mantendremos las puertas abiertas, como hasta ahora, y también os animo a ver esta página como un punto de encuentro privilegiado y de difusión de nuevos proyectos e iniciativas.</seg>
      </tuv>
    </tu>
    <tu tuid="22960">
      <tuv xml:lang="en">
        <seg>A problem as occured: your browser is not configured to accept iframes</seg>
      </tuv>
      <tuv xml:lang="es">
        <seg>Acceda aquí a la página web de la Mostra Portuguesa 2015:</seg>
      </tuv>
    </tu>
  </body>
</tmx>
"""

# Parsear el XML
# Recrear el árbol XML desde los datos proporcionados
root = ET.fromstring(xml_data)

# Reiniciar el archivo CSV en memoria
csv_file = io.StringIO()
csv_writer = csv.writer(csv_file, delimiter='~')
csv_writer.writerow(['english', 'spanish'])  # Escribir cabeceras

# Procesar cada <tu> en el XML
for tu in root.findall('.//tu'):
    english_text, spanish_text = None, None

    # Buscar las oraciones en inglés y español
    for tuv in tu.findall('.//tuv'):
        lang = tuv.get('{http://www.w3.org/XML/1998/namespace}lang')
        seg_text = tuv.find('seg').text
        if lang == 'en':
            english_text = seg_text
        elif lang == 'es':
            spanish_text = seg_text

    # Escribir las oraciones en el CSV solo si ambas están presentes
    if english_text and spanish_text:
        csv_writer.writerow([english_text, spanish_text])

# Mover el puntero al inicio del archivo para leerlo
csv_file.seek(0)

# Leer y mostrar las líneas del archivo CSV generado
csv_content = csv_file.read()
csv_content

# Guardar el archivo CSV generado
with open('./datasets/dataset_web.csv', 'w', encoding='utf-8') as f:
    f.write(csv_content)


# Scripts para descargar datasets de OPUS: 

https://github.com/Helsinki-NLP/Opus-MT-train

Para descargarnos datasets bilingues español-inglés de OPUS para entrenar modelos de traducción.

In [1]:
# %pip install opustools-pkg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 1.7 MB/s eta 0:00:00a 0:00:01
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


(NOTA: Al final me he creado mis propios scripts que parsean los xml en crudo) Preprocesamiento del dataset de OPUS una vez descargado:

https://github.com/bitextor/bitextor

- bifixer: Los datos se preprocesan con Bifixer, una herramienta para limpiar y filtrar corpus paralelos. Bifixer identifica y arregla errores comunes en los corpus paralelos, como oraciones que no están alineadas correctamente.

- bicleaner: Los datos se preprocesan con Bicleaner, una herramienta para clasificar y filtrar pares de oraciones en corpus paralelos según la probabilidad de que sean una traducción válida una de la otra.

- bifixer + bicleaner: aplicar ambos filtros

## descargar Dataset de OPUS: libros en español e inglés

In [3]:
# !opus_get -d Books -s es -t en -p raw
!opus_get -d Books -s es -t en -p parsed

usage: opus-get [-h] [-s SOURCE] [-t TARGET] [-d DIRECTORY] [-r RELEASE]
                [-p {raw,xml,parsed}] [-l] [-dl DOWNLOAD_DIR] [-q]
opus-get: error: argument -l/--list_resources: ignored explicit argument 'n'


### Crear csv con frases de libros en español e inglés

#### Primero vamos a contar las letras de la novela

In [2]:
# Primero, necesito leer el archivo XML para entender su estructura y localizar las etiquetas relevantes.
import xml.etree.ElementTree as ET

# Ruta del archivo
file_path = 'datasets/opus_books_es_en/Books/raw/es/Verne_Jules-Ile_mysterieuse.xml'

# Cargar el archivo XML
tree = ET.parse(file_path)
root = tree.getroot()

# Imprimiendo los primeros elementos para entender la estructura
for element in root.iter():
    print(element.tag, element.attrib)
    if len(list(element)) == 0:  # Si el elemento no tiene subelementos, imprime su texto
        print(element.text)
    if element.tag == 's':
        break  # Solo queremos ver hasta la primera etiqueta 's' para entender la estructura


text {}
head {}
meta {}
 The Mysterious Island 
 by Jules Verne 
 Aligned by: András Farkas (autoalignment) 
 
body {}
s {'id': 's1'}
Source: http://librosgratis.liblit.com/Translation: Joaquín Gallardo (?)


In [10]:
import string

# Función para contar las letras en un texto, excluyendo espacios y signos de puntuación
def count_letters(text):
    return sum(1 for char in text if char.isalpha())

# Función para contar todas las letras de un texto (incluidas espacios y signos de puntuación)
def count_all_letters(text):
    return sum(1 for char in text if char in string.printable)

# Contador total de letras
total_letters = 0

total_frase = 0
# contar número de elementos 's'
for elem in root.iter('s'):
    total_frase += 1

# Iterar a través de los elementos 's' y contar las letras
for elem in root.iter('s'):
    if elem.text:
        total_letters += count_letters(elem.text)

print(total_letters)

# Contador total de letras
total_letters = 0
for elem in root.iter('s'):
    if elem.text:
        total_letters += count_all_letters(elem.text)

print(total_letters)

print ("total de frases:", total_frase)

print("media de letras por frase:", total_letters / total_frase)


918619
1105657
total de frases: 9990
media de letras por frase: 110.67637637637638


#### Script para crear grupos de frases, genera una lista de grupos que es una lista de  pares:
```python
[(id_primera_frase_grupo, concatenación_de_frases_del_grupo), (id, frases), ...]

In [18]:
import xml.etree.ElementTree as ET

def generate_text_groups(xml_path):
    """
    Procesa un archivo XML y devuelve una lista de pares (id, texto acumulado),
    agrupando las frases hasta alcanzar un límite de 4000 caracteres, y cerrando
    el grupo al finalizar un párrafo si es necesario.

    :param xml_path: Ruta del archivo XML
    :return: Lista de pares (id, texto acumulado)
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Lista para almacenar los pares (id, texto)
    text_groups = []
    es_parrafo = False
    parrafo_id = None
    current_text = ""  # Texto acumulado actual
    group_start_id = None  # ID de inicio del grupo actual
    limite_base = 1000  # Límite de caracteres para cada grupo

    for elem in root.iter():
        # Revisar si es un elemento 's' (frase)
        if elem.tag == 's' and 'id' in elem.attrib:
            id = elem.attrib['id']
            era_parrafo = es_parrafo
            es_parrafo = "." in id

            # identificamos el parrafo actual
            parrafo_anterior = parrafo_id
            punto = id.find(".")
            # los id son de la forma: id="s1" o id="s2" quitamos la s y convertimos a int
            if punto == -1:
                parrafo_id = int(id[1:])
            else:
                parrafo_id = int(id[1:punto])

            
            if era_parrafo and len(current_text) >= limite_base and (not es_parrafo or parrafo_anterior != parrafo_id):
                # Cerrar el grupo al final del párrafo actual
                text_groups.append((group_start_id, current_text.strip()))
                current_text = ""
                group_start_id = None

            if group_start_id is None:            
                # vemos donde empieza el punto "."
                punto = id.find(".")
                # los id son de la forma: id="s1" o id="s2" quitamos la s y convertimos a int
                if punto == -1:
                    group_start_id = int(id[1:])
                else:
                    group_start_id = int(id[1:punto])
                # si es de la forma id="s1.1" o id="s1.2" entonces estamos dentro de un párrafo
                
                

                # group_start_id = int(elem.attrib['id'])  # Establecer el ID inicial del grupo

            # Agregar texto de la frase actual
            if elem.text:
                current_text += elem.text + " "  # Añadir un espacio entre frases

            # Revisar si se alcanzó o superó el límite de caracteres
            if len(current_text) >= limite_base and not es_parrafo:
                # Cerrar el grupo al final del párrafo actual
                text_groups.append((group_start_id, current_text.strip()))
                current_text = ""
                group_start_id = None

        # Cerrar el grupo si se terminó el documento
        # if elem.tag == 'body':
        #     if current_text:
        #         text_groups.append((group_start_id, current_text.strip()))
    
    if current_text:
        text_groups.append((group_start_id, current_text.strip()))

    return text_groups

# Ruta del archivo XML
file_path = 'datasets/opus_books_es_en/Books/raw/es/Austen_Jane-Sense_and_Sensibility.xml'

# Probando la función con el archivo XML dado
text_groups = generate_text_groups(file_path)
# text_groups[:10]  # Mostrar los primeros 3 grupos para verificar el resultado


In [15]:
len(text_groups) # Mostrar los últimos 3 grupos para verificar el resultado

245

In [19]:
text_groups[0]

(1,
 'Source: Wikisource & librodot.com SENTIDO Y SENSIBILIDAD JANE AUSTEN CAPITULO I La familia Dashwood llevaba largo tiempo afincada en Sussex. Su propiedad era de buen tamaño, y en el centro de ella se encontraba la residencia, Norland Park, donde la manera tan digna en que habían vivido por muchas generaciones llegó a granjearles el respeto de todos los conocidos del lugar. El último dueño de esta propiedad había sido un hombre soltero, que alcanzó una muy avanzada edad, y que durante gran parte de su existencia tuvo en su hermana una fiel compañera y ama de casa. Pero la muerte de ella, ocurrida diez años antes que la suya, produjo grandes alteraciones en su hogar. Para compensar tal pérdida, invitó y recibió en su casa a la familia de su sobrino, el señor Henry Dashwood, el legítimo heredero de la finca Norland y la persona a quien se proponía dejarla en su testamento. En compañía de su sobrino y sobrina, y de los hijos de ambos, la vida transcurrió confortablemente para el anci

In [4]:
text_groups[-2:]

[(10317,
  'Una sola perla, la más hermosa, fue separada de aquel tesoro y enviada a lady Glenarvan, en nombre de los náufragos devueltos por el Duncan a su patria. En aquella propiedad, los colonos llamaron al trabajo, es decir, a la fortuna y a la felicidad, a todos aquellos a quienes pensaban ofrecer hospitalidad en la isla Lincoln. Se fundó una gran colonia a la cual dieron el nombre de la isla que había desaparecido en las profundidades del Pacífico. Se veía un río que se llamaba de la Merced, un monte que tomó el nombre de Franklin, un pequeño lago que fue el lago Grant y bosques que recibieron la denominación de Far–West. Era como una isla en tierra firme. En ella, bajo la mano inteligente del ingeniero y de sus compañeros, todo prosperó. Ni uno solo de los antiguos colonos de la isla Lincoln faltaba, porque habían jurado vivir siempre juntos. Nab estaba siempre donde su amo; Ayrton, dispuesto a sacrificarse en toda ocasión; Pencroff, más labrador que marino; Harbert terminó sus

In [5]:
text_groups
ids = [pair[0] for pair in text_groups]
ids[:10]

[1, 17, 22, 32, 40, 49, 56, 67, 80, 93]

#### Parsear el xml del otro idioma

In [6]:
import xml.etree.ElementTree as ET
import sys

def parse_english_xml(xml_path, start_ids):
    """
    Parse the English XML file and group the sentences based on the start IDs.

    :param xml_path: Path of the English XML file
    :param start_ids: List of start IDs for grouping
    :return: List of pairs (group_id, concatenated_text)
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    text_groups = []
    current_text = ""
    group_start_id = start_ids.pop(0)
    next_group_start_id = start_ids.pop(0) if start_ids else sys.maxsize

    for elem in root.iter():
        if elem.tag == 's' and 'id' in elem.attrib:
            id = elem.attrib['id']
            
            punto = id.find(".")
            # los id son de la forma: id="s1" o id="s2" quitamos la s y convertimos a int
            if punto == -1:
                peer_group_start_id = int(id[1:])
            else:
                peer_group_start_id = int(id[1:punto])


            if  peer_group_start_id < next_group_start_id:
                if elem.text:
                    current_text += elem.text + " "
            else:
                text_groups.append((group_start_id, current_text.strip()))
                current_text = ""
                group_start_id = next_group_start_id
                next_group_start_id = start_ids.pop(0) if start_ids else sys.maxsize

                if elem.text:
                    current_text += elem.text + " "
                    
        # if elem.tag == 'body':
        #     if current_text:
        #         text_groups.append((group_start_id, current_text.strip()))
                # current_text = ""
                # if start_ids:
                #     group_start_id = start_ids.pop(0)
    # Cerrar el grupo si se terminó el documento
    if current_text:
        text_groups.append((group_start_id, current_text.strip()))

    return text_groups

#
english_xml_path = 'datasets/opus_books_es_en/Books/raw/en/Verne_Jules-Ile_mysterieuse.xml'

# Start ID for grouping
# start_id = text_groups[0][0]

ids = [pair[0] for pair in text_groups]

# Parse the English XML file and group the sentences
english_text_groups = parse_english_xml(english_xml_path, ids)

# Show the first 3 groups for verification
# english_text_groups[:3]
# ids

In [7]:
english_text_groups[-2:]

[(10317,
  'One pearl alone, the finest, was reserved from the treasure and sent to Lady Glenarvan in the name of the castaways restored to their country by the "Duncan." There, upon this domain, the colonists invited to labor, that is to say, to wealth and happiness, all those to whom they had hoped to offer the hospitality of Lincoln Island. There was founded a vast colony to which they gave the name of that island sunk beneath the waters of the Pacific. A river there was called the Mercy, a mountain took the name of Mount Franklin, a small lake was named Lake Grant, and the forests became the forests of the Far West. It might have been an island on terra firma. There, under the intelligent hands of the engineer and his companions, everything prospered. Not one of the former colonists of Lincoln Island was absent, for they had sworn to live always together. Neb was with his master; Ayrton was there ready to sacrifice himself for all; Pencroft was more a farmer than he had ever been a

#### Generar csv con frases de libros en español e inglés

In [8]:
import csv

# Combine Spanish and English text groups
combined_groups = zip(text_groups, english_text_groups)

# Define the CSV file path
csv_file_path = 'text_groups.csv'

# Write the combined groups to the CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file, delimiter='~')  # Set the delimiter to "~"
    writer.writerow(['Spanish', 'English'])  # Write the header row
    for (spanish_group, english_group) in combined_groups:
        writer.writerow([spanish_group[1], english_group[1]])


In [10]:
import pandas as pd
# Read the CSV file
df = pd.read_csv('text_groups.csv', delimiter='~')

# Print the last 2 records without the 'sep' argument
print(df.tail(2).to_string(index=False))


#### Script para recorrer todos los xml en español y crear un csv con los grupos de frases de cada libro

In [20]:
import os
import csv

# Define the paths
xml_folder = 'datasets/opus_books_es_en/Books/raw/es'
csv_folder = 'datasets/opus_books_es_en/Books/csv/'

# Create the csv folder if it doesn't exist
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)

# Iterate through the XML files
for file_name in os.listdir(xml_folder):
    if file_name.endswith('.xml'):
        # Construct the file paths
        xml_file_path = os.path.join(xml_folder, file_name)
        csv_file_path = os.path.join(csv_folder, file_name.replace('.xml', '.csv'))

        # Parse the XML file and generate the text groups
        text_groups = generate_text_groups(xml_file_path)

        # Parse the corresponding English XML file and generate the English text groups
        english_xml_file_path = os.path.join('datasets/opus_books_es_en/Books/raw/en', file_name)
        english_text_groups = parse_english_xml(english_xml_file_path, [pair[0] for pair in text_groups])

        # Combine the Spanish and English text groups
        combined_groups = zip(text_groups, english_text_groups)

        # Write the combined groups to the CSV file
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file, delimiter='~')
            writer.writerow(['Spanish', 'English'])
            for (spanish_group, english_group) in combined_groups:
                writer.writerow([spanish_group[1], english_group[1]])

        print(f"CSV file created: {csv_file_path}")


CSV file created: datasets/opus_books_es_en/Books/csv/Austen_Jane-Sense_and_Sensibility.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Bronte_Charlotte-Jane_Eyre.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Carroll_Lewis-Alice_in_wonderland.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Cervantes_Miguel-Don_Quijote.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Defoe_Daniel-Robinson_Crusoe.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Doyle_Arthur_Conan-Adventures_of_Sherlock_Holmes.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Doyle_Arthur_Conan-A_Study_in_Scarlet.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Doyle_Arthur_Conan-Hound_of_the_Baskervilles.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Dumas_Alexandre-Trois_Mousquetaires.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Kafka_Franz-Verwandlung.csv
CSV file created: datasets/opus_books_es_en/Books/csv/Poe_Edgar_Allan-Fall_

Eliminamos Don Quijote por tener un lenguaje demasiado antiguo y no ser representativo de un lenguaje moderno.

In [21]:
import os

# Define the directory where the .csv files are located
directory = 'datasets/opus_books_es_en/Books/csv'

# Iterate through the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv') and 'Don_Quijote' in filename:
        file_path = os.path.join(directory, filename)
        os.remove(file_path)
        print(f"File deleted: {file_path}")



File deleted: datasets/opus_books_es_en/Books/csv/Cervantes_Miguel-Don_Quijote.csv


### concatenamos todos los .csv en un unico .csv dataset

In [13]:
%%bash
# Crear un nuevo archivo con la cabecera deseada
echo "Spanish~English" > opus_books_dataset.csv

# Concatenar todos los archivos omitiendo su primera línea
for file in datasets/opus_books_es_en/Books/csv/*.csv; do
    tail -n +2 "$file" >> opus_books_dataset.csv
done


## descargar Dataset de TED2020 OPUS:

In [ ]:
# Este dataset contiene transcripciones y traducciones de las charlas TED hasta el año 2020. Es útil para tareas que requieren un lenguaje más formal y académico.
!opus_get -d TED2020 -s es -t en -p raw

### Parsear xml de TED2020 y crear una lista de diccionarios con las secciones cruzadas de los idiomas

Función que toma como entrada el xml de cruces de idiomas de TED2020 y devuelve una lista de diccionarios con las secciones cruzadas de los idiomas de esta forma:
```python
 [{"fromDoc" : "en/ted2020-1.xml", "toDoc" : "es/ted2020-1.xml", "sections" : [{"EN" : 1, "SP" : 1}, {"EN" : 2, "SP" : 2}, {"EN" : 3, "SP" : 3}, ... {"EN" : 142, "SP" : 138}]}, {"fromDoc" : "en/ted2020-10.xml", "toDoc" : "es/ted2020-10.xml", "sections" : [{"EN" : 1, "SP" : 1}, {"EN" : 2, "SP" : 2}, {"EN" : 3, "SP" : 3}, ... {"EN" : 13, "SP" : 13}]} , ... ]
```

In [2]:
import xml.etree.ElementTree as ET
import sys

def parse_xml_to_dict(xml_file):
    # Read the XML file
    with open(xml_file, 'r') as file:
        xml_content = file.read()

    # Remove leading whitespace
    xml_content = xml_content.strip()

    # Parse the XML content
    root = ET.fromstring(xml_content)

    # Initialize the list to hold all link groups
    link_groups = []

    # Iterate over each linkGrp element
    for linkGrp in root.findall('linkGrp'):
        # Extract fromDoc and toDoc
        from_doc = linkGrp.attrib['fromDoc']
        to_doc = linkGrp.attrib['toDoc']

        # Initialize the list to hold all sections
        sections = []

        # Iterate over each link element within the linkGrp
        for link in linkGrp.findall('link'):
            # Get xtargets attribute
            xtargets = link.attrib['xtargets'].split(';')
            
            # Handling multiple sections
            en_sections = xtargets[0].split()
            sp_sections = xtargets[1].split()

            # Solo marcamos los enlaces que hacen de tope claro (tienen referencia uno a uno) todo lo anterior se acumularía en el mismo grupo cuando se procese este objeto.
            if len(en_sections) == 1 and len(sp_sections) == 1:
                sections.append({"EN": int(float(en_sections[0])), "SP": int(float(sp_sections[0]))})
            # for i, en_section in enumerate(en_sections):
            #     try:
            #         sp_section = sp_sections[min(i, len(sp_sections) - 1)]
            #     except IndexError:
            #         print(f"IndexError: {en_sections}, {sp_sections} #frondoc {from_doc}, #todoc: {to_doc}")
            #         continue
            #     sections.append({"EN": int(float(en_section)), "SP": int(float(sp_section))})

        # Add the link group to the list
        link_groups.append({"fromDoc": from_doc, "toDoc": to_doc, "sections": sections})

    return link_groups

# xml_file = 'datasets/TED2020/TED2020_latest_xml_en-es.xml'
xml_file = 'datasets/News-Commentary/News-Commentary_latest_xml_en-es.xml'
result = parse_xml_to_dict(xml_file)
# print(result[0])


import pickle

# Guardar el objeto diccionario en un archivo usando pickle
with open('result.pkl', 'wb') as file:
    pickle.dump(result, file)  #'wb' indica que el archivo se abre en modo escritura binaria

In [3]:
result[0]

{'fromDoc': 'en/-10-000-gold.xml.gz',
 'toDoc': 'es/-10-000-gold.xml.gz',
 'sections': [{'EN': 1, 'SP': 1},
  {'EN': 2, 'SP': 2},
  {'EN': 3, 'SP': 3},
  {'EN': 4, 'SP': 4},
  {'EN': 5, 'SP': 5},
  {'EN': 6, 'SP': 6},
  {'EN': 7, 'SP': 7},
  {'EN': 8, 'SP': 8},
  {'EN': 9, 'SP': 9},
  {'EN': 10, 'SP': 10},
  {'EN': 11, 'SP': 11},
  {'EN': 12, 'SP': 12},
  {'EN': 13, 'SP': 13},
  {'EN': 14, 'SP': 14},
  {'EN': 15, 'SP': 15},
  {'EN': 16, 'SP': 16},
  {'EN': 17, 'SP': 17},
  {'EN': 18, 'SP': 18},
  {'EN': 19, 'SP': 19},
  {'EN': 20, 'SP': 20},
  {'EN': 21, 'SP': 21},
  {'EN': 22, 'SP': 22},
  {'EN': 23, 'SP': 23},
  {'EN': 24, 'SP': 24},
  {'EN': 25, 'SP': 25},
  {'EN': 26, 'SP': 26},
  {'EN': 27, 'SP': 27},
  {'EN': 28, 'SP': 28},
  {'EN': 29, 'SP': 29},
  {'EN': 30, 'SP': 30},
  {'EN': 31, 'SP': 31},
  {'EN': 32, 'SP': 32},
  {'EN': 33, 'SP': 33},
  {'EN': 34, 'SP': 34},
  {'EN': 35, 'SP': 35},
  {'EN': 36, 'SP': 36},
  {'EN': 37, 'SP': 37},
  {'EN': 38, 'SP': 38},
  {'EN': 39, 'SP': 3

#### guardar objeto en formato pickle y recuperarlo

In [3]:
import pickle

# Guardar el objeto diccionario en un archivo usando pickle
with open('result.pkl', 'wb') as file:
    pickle.dump(result, file)  #'wb' indica que el archivo se abre en modo escritura binaria

In [4]:
import pickle

# Load the cross-language dictionary from the pickle file
with open('result.pkl', 'rb') as file:
    cross_language_dict = pickle.load(file)

In [5]:
cross_language_dict[:3]

[{'fromDoc': 'en/-10-000-gold.xml.gz',
  'toDoc': 'es/-10-000-gold.xml.gz',
  'sections': [{'EN': 1, 'SP': 1},
   {'EN': 2, 'SP': 2},
   {'EN': 3, 'SP': 3},
   {'EN': 4, 'SP': 4},
   {'EN': 5, 'SP': 5},
   {'EN': 6, 'SP': 6},
   {'EN': 7, 'SP': 7},
   {'EN': 8, 'SP': 8},
   {'EN': 9, 'SP': 9},
   {'EN': 10, 'SP': 10},
   {'EN': 11, 'SP': 11},
   {'EN': 12, 'SP': 12},
   {'EN': 13, 'SP': 13},
   {'EN': 14, 'SP': 14},
   {'EN': 15, 'SP': 15},
   {'EN': 16, 'SP': 16},
   {'EN': 17, 'SP': 17},
   {'EN': 18, 'SP': 18},
   {'EN': 19, 'SP': 19},
   {'EN': 20, 'SP': 20},
   {'EN': 21, 'SP': 21},
   {'EN': 22, 'SP': 22},
   {'EN': 23, 'SP': 23},
   {'EN': 24, 'SP': 24},
   {'EN': 25, 'SP': 25},
   {'EN': 26, 'SP': 26},
   {'EN': 27, 'SP': 27},
   {'EN': 28, 'SP': 28},
   {'EN': 29, 'SP': 29},
   {'EN': 30, 'SP': 30},
   {'EN': 31, 'SP': 31},
   {'EN': 32, 'SP': 32},
   {'EN': 33, 'SP': 33},
   {'EN': 34, 'SP': 34},
   {'EN': 35, 'SP': 35},
   {'EN': 36, 'SP': 36},
   {'EN': 37, 'SP': 37},
   {'

In [6]:
len(cross_language_dict)

10680

### Script para, a partir del objeto diccionario cruzado, recorrer todos los xml de TED2020 y crear un csv con las secciones cruzadas de los idiomas. 

Agrupamos lineas de forma aleatoria para que el modelo de traducción tenga más contexto variado.

In [10]:
import os
import csv
import xml.etree.ElementTree as ET
import pickle
import random

# Load the cross-language dictionary from the pickle file
with open('result.pkl', 'rb') as file:
    cross_language_dict = pickle.load(file)

# Define the paths
xml_sp_folder = './datasets/News-Commentary/News-Commentary/raw/es'
xml_en_folder = './datasets/News-Commentary/News-Commentary/raw/en'
csv_file_path = 'cross_language_sections.csv'

# Create the CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file, delimiter='~')
    writer.writerow(['From Document', 'To Document', 'EN Section', 'SP Section'])

    # Iterate through the cross-language dictionary
    for entry in cross_language_dict:
        from_doc = entry['fromDoc']
        from_doc = os.path.splitext(os.path.basename(from_doc))[0]
        to_doc = entry['toDoc']
        to_doc = os.path.splitext(os.path.basename(to_doc))[0]

        xml_en_file_path = os.path.join(xml_en_folder, from_doc)
        xml_sp_file_path = os.path.join(xml_sp_folder, to_doc)

        try:
            tree_en = ET.parse(xml_en_file_path)
            root_en = tree_en.getroot()
        except:
            print(f"Error: {xml_en_file_path} not found")
            continue

        try:
            tree_sp = ET.parse(xml_sp_file_path)
            root_sp = tree_sp.getroot()
        except:
            print(f"Error: {xml_sp_file_path} not found")
            continue

        sections = entry['sections']

        en_anterior = 0
        sp_anterior = 0
        while sections:
            # section = sections.pop(0)
            # en_from_section = section['EN']
            # sp_from_section = section['SP']


            # Determinar cuántos elementos avanzar, con un máximo del tamaño de la lista restante
            advance_steps = min(random.randint(1, 8), len(sections))

            # Avanzar en la lista y obtener la sección de destino
            for _ in range(advance_steps):
                if sections:
                    section = sections.pop(0)

            en_to_section = section['EN'] if sections else float('inf')
            sp_to_section = section['SP'] if sections else float('inf')

            en_text = ""
            for elem in root_en.iter('s'):
                if elem.text:
                    try:
                        id = int(float(elem.attrib['id']))
                    except:
                        print(f"Error: no se pudo convertir a int el id {id} que viene de {elem.attrib['id']}")
                        continue
                    if en_anterior <= id < en_to_section:
                        en_text += elem.text + " "

            sp_text = ""
            for elem in root_sp.iter('s'):
                if elem.text:
                    try:
                        id = int(float(elem.attrib['id']))
                    except:
                        print(f"Error: no se pudo convertir a int el id {id} que viene de {elem.attrib['id']}")
                        continue
                    if sp_anterior <= id < sp_to_section:
                        sp_text += elem.text + " "

            writer.writerow([from_doc, to_doc, en_text.strip(), sp_text.strip()])
            en_anterior = en_to_section
            sp_anterior = sp_to_section

print(f"CSV file created: {csv_file_path}")


Error: ./datasets/News-Commentary/News-Commentary/raw/en/-the-dictatorship-of-values.xml not found
Error: ./datasets/News-Commentary/News-Commentary/raw/en/a-century-of-chinese-protests.xml not found
Error: ./datasets/News-Commentary/News-Commentary/raw/en/a-civilization-in-crisis.xml not found
Error: ./datasets/News-Commentary/News-Commentary/raw/en/america-the-pariah.xml not found
Error: ./datasets/News-Commentary/News-Commentary/raw/en/an-agenda-for-the-group-of-7.xml not found
Error: ./datasets/News-Commentary/News-Commentary/raw/en/argentina-and-the-soft-underbelly-of-hard-money.xml not found
Error: ./datasets/News-Commentary/News-Commentary/raw/en/avoiding-the-terrorists--trap.xml not found
Error: ./datasets/News-Commentary/News-Commentary/raw/en/backing-the-right-horse-in-pakistan.xml not found
Error: ./datasets/News-Commentary/News-Commentary/raw/en/beijing-benefits-from-taiwan-s-troubles.xml not found
Error: ./datasets/News-Commentary/News-Commentary/raw/en/berlusconi-s-silent

## Descargar dataset de OPUS: noticias

In [ ]:
!opus_get -d News-Commentary -s es -t en raw

## Descargar dataset de películas y series de TV de OPUS

In [ ]:
!opus_get -d OpenSubtitles -s es -t en -p raw

### Parseamos xml de cruces y creamos un diccionario con las secciones cruzadas de los idiomas específico de este dataset

Lo parseamos con ET.interparse para que no de error de memoria al parsear el xml

In [3]:
import xml.etree.ElementTree as ET
import pickle

def parse_xml_to_dict_iterparse(xml_file):
    link_groups = []

    contador = 0
    for event, elem in ET.iterparse(xml_file, events=("start", "end")):
        if event == 'start':
            if elem.tag == 'linkGrp':
                from_doc = elem.attrib['fromDoc']
                to_doc = elem.attrib['toDoc']
                sections = []
            elif elem.tag == 'link':
                xtargets = elem.attrib['xtargets'].split(';')
                overlap = elem.attrib.get('overlap', 0)

                en_sections = xtargets[0].split()
                sp_sections = xtargets[1].split()

                # if len(en_sections) == 1 and len(sp_sections) == 1 and ((float(overlap)) > 0.99 or contador > 3 and (float(overlap)) > 0.95 or contador > 6 and (float(overlap)) > 0.9):
                # solo cojo como inicio de sección las secciones bien alineadas y que no se solapen
                if len(en_sections) == 1 and len(sp_sections) == 1 and (float(overlap)) > 0.99:
                    sections.append({"EN": int(float(en_sections[0])), "SP": int(float(sp_sections[0]))})
                    contador = 0
                else:
                    contador += 1

        elif event == 'end' and elem.tag == 'linkGrp':
            link_groups.append({"fromDoc": from_doc, "toDoc": to_doc, "sections": sections})
            elem.clear()

    return link_groups

xml_file = 'datasets/OpenSubtitles/OpenSubtitles_latest_xml_en-es.xml'
result = parse_xml_to_dict_iterparse(xml_file)

# Guardar el objeto diccionario en un archivo usando pickle
with open('resultPelis.pkl', 'wb') as file:
    pickle.dump(result, file)  #'wb' indica que el archivo se abre en modo escritura binaria

In [18]:
import pickle

# Load the cross-language dictionary from the pickle file
with open('resultPelis.pkl', 'rb') as file:
    cross_language_dict = pickle.load(file)

#### Haciendo pruebas con el dataset de películas y series de TV de OPUS. Filtro y Me quedo solo con un fichero para debugear problemas.

In [19]:
# Filtrar para obtener solo los elementos con fromDoc específico
filtered_data = [item for item in cross_language_dict if item['fromDoc'] == 'en/2010/1385867/3699446.xml.gz']
filtered_data

[{'fromDoc': 'en/2010/1385867/3699446.xml.gz',
  'toDoc': 'es/2010/1385867/3698427.xml.gz',
  'sections': [{'EN': 1, 'SP': 2},
   {'EN': 4, 'SP': 5},
   {'EN': 5, 'SP': 6},
   {'EN': 13, 'SP': 14},
   {'EN': 19, 'SP': 20},
   {'EN': 24, 'SP': 25},
   {'EN': 26, 'SP': 27},
   {'EN': 27, 'SP': 28},
   {'EN': 33, 'SP': 34},
   {'EN': 41, 'SP': 41},
   {'EN': 45, 'SP': 45},
   {'EN': 47, 'SP': 50},
   {'EN': 56, 'SP': 59},
   {'EN': 61, 'SP': 63},
   {'EN': 66, 'SP': 68},
   {'EN': 67, 'SP': 69},
   {'EN': 72, 'SP': 74},
   {'EN': 81, 'SP': 82},
   {'EN': 82, 'SP': 83},
   {'EN': 90, 'SP': 92},
   {'EN': 94, 'SP': 96},
   {'EN': 99, 'SP': 101},
   {'EN': 100, 'SP': 102},
   {'EN': 102, 'SP': 104},
   {'EN': 107, 'SP': 109},
   {'EN': 110, 'SP': 112},
   {'EN': 117, 'SP': 118},
   {'EN': 118, 'SP': 119},
   {'EN': 128, 'SP': 126},
   {'EN': 131, 'SP': 129},
   {'EN': 135, 'SP': 133},
   {'EN': 136, 'SP': 134},
   {'EN': 144, 'SP': 142},
   {'EN': 148, 'SP': 146},
   {'EN': 153, 'SP': 150},


In [20]:
# Guardar el objeto diccionario en un archivo usando pickle
with open('filterData.pkl', 'wb') as file:
    pickle.dump(filtered_data, file)  #'wb' indica que el archivo se abre en modo escritura binaria

In [ ]:
with open('filterData.pkl', 'rb') as file:
    cross_language_dict = pickle.load(file)

cross_language_dict

### Script para, a partir del objeto diccionario cruzado, recorrer todos los xml de películas y series de TV y crear un csv con las secciones cruzadas de los idiomas.

In [4]:
%%writefile parse_xml_OPUS_pelis_to_csv.py
import os
import csv
import xml.etree.ElementTree as ET
import pickle
import random

# Load the cross-language dictionary from the pickle file
with open('resultPelis.pkl', 'rb') as file:
    cross_language_dict = pickle.load(file)

# Define the paths
xml_es_folder = './datasets/OpenSubtitles/OpenSubtitles_latest_raw_es/OpenSubtitles/raw/'
xml_en_folder = './datasets/OpenSubtitles/OpenSubtitles_latest_raw_en/OpenSubtitles/raw/'
csv_file_path = 'OpenSubtitlesDataset.csv'

# Función para extraer el texto más largo ya que a veces hay texto en el elemento y a veces en el tail
def extract_longest_text(element):
    longest_text = ""
    if element.text and len(element.text.strip()) > len(longest_text):
        longest_text = element.text.strip()
    for child in element:
        if child.tail and len(child.tail.strip()) > len(longest_text):
            longest_text = child.tail.strip()
    return longest_text

# Create the CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file, delimiter='~')
    writer.writerow(['From Document', 'To Document', 'EN Section', 'SP Section'])

    for entry in cross_language_dict:
        from_doc = os.path.splitext(entry['fromDoc'])[0]
        to_doc = os.path.splitext(entry['toDoc'])[0]

        xml_en_file_path = os.path.join(xml_en_folder, from_doc)
        xml_sp_file_path = os.path.join(xml_es_folder, to_doc)

        try:
            tree_en = ET.parse(xml_en_file_path)
            root_en = tree_en.getroot()
        except:
            print(f"Error: {xml_en_file_path} not found")
            continue

        try:
            tree_sp = ET.parse(xml_sp_file_path)
            root_sp = tree_sp.getroot()
        except:
            print(f"Error: {xml_sp_file_path} not found")
            continue

        sections = entry['sections']

        en_anterior = 0
        sp_anterior = 0
        while sections:
            advance_steps = min(random.randint(1, 8), len(sections))

            for _ in range(advance_steps):
                if sections:
                    section = sections.pop(0)

            en_to_section = section['EN'] if sections else float('inf')
            sp_to_section = section['SP'] if sections else float('inf')

            en_text = ""
            for elem in root_en.iter('s'):
                if int(float(elem.attrib['id'])) >= en_anterior and int(float(elem.attrib['id'])) < en_to_section:
                    en_text += extract_longest_text(elem) + " "

            sp_text = ""
            for elem in root_sp.iter('s'):
                if int(float(elem.attrib['id'])) >= sp_anterior and int(float(elem.attrib['id'])) < sp_to_section:
                    sp_text += extract_longest_text(elem) + " "

            # si la longitud del texto es menor que 2000 caracteres no lo añadimos (eliminamos textos muy largos fruto de la concatenación de varios textos que no se han podido alinear bien)
            if len(en_text) < 2000 and len(sp_text) < 2000:
                writer.writerow([from_doc, to_doc, en_text.strip(), sp_text.strip()])
            en_anterior = en_to_section
            sp_anterior = sp_to_section

print(f"CSV file created: {csv_file_path}")


Overwriting parse_xml_OPUS_pelis_to_csv.py


## Descargar dataset de OPUS: Naciones Unidas.

In [ ]:
#Este dataset contiene documentos y transcripciones de las Naciones Unidas. Es útil para tareas que requieren un lenguaje formal y conocimiento de temas internacionales y diplomáticos.
!opus_get -d MultiUN -s es -t en -p raw

## Descargamos dataset parlamento europeo

In [ ]:
#Este dataset contiene las transcripciones de los debates del Parlamento Europeo. Es útil para tareas que requieren un lenguaje formal y conocimiento de temas políticos y legislativos.
!opus_get -d Europarl -s es -t en -p raw

## Descargamos dataset de OPUS: wikipedia

In [ ]:
!opus_get -d Wikipedia -s es -t en -p raw

# Traducciones de calidad español Inglés

Comprobamos el excel de traducciones de calidad.

In [4]:
import pandas as pd

# Cargar el archivo Excel
file_path = './datasets/Traslate_spanish_english.xlsx'
excel_data = pd.read_excel(file_path)

# Mostrar las primeras filas para entender la estructura del archivo
excel_data.head()

,Está en la mesa.,It's on the table.
0,Él es profesor.,He's a teacher.
1,Ella es alta.,She's tall.
2,No sé.,I don't know.
3,No está aquí.,It's not here.
4,Soy de España.,I'm from Spain.


Pasar a formato .csv

In [5]:
# Asignar nombres de columna
excel_data.columns = ["Spanish", "English"]

# Ruta para el archivo CSV de salida
output_csv_path = './datasets/Traslate_spanish_english.csv'

# Guardar el dataframe en formato CSV con el separador '~'
excel_data.to_csv(output_csv_path, sep='~', index=False)

output_csv_path


'./datasets/Traslate_spanish_english.csv'

In [2]:
import pandas as pd

# Ruta para el archivo CSV de salida
output_csv_path = './datasets/Traslate_spanish_english.csv'

# Leer los primeros 100 registros del archivo CSV
csv_data = pd.read_csv(output_csv_path, sep='~') #, nrows=100)
csv_data.head()

,Spanish,English
0,Él es profesor.,He's a teacher.
1,Ella es alta.,She's tall.
2,No sé.,I don't know.
3,No está aquí.,It's not here.
4,Soy de España.,I'm from Spain.


In [4]:
import pandas as pd

# Suponiendo que output_csv_path es la ruta al archivo CSV
output_csv_path = './datasets/Traslate_spanish_english.csv'

# Cargar el archivo CSV
dataset = pd.read_csv(output_csv_path, delimiter='~')

# Encontrar los índices donde la columna "spanish" está vacía
indices_vacios = dataset[dataset['Spanish'].isna() | (dataset['Spanish'] == '')].index

print(indices_vacios.tolist())


[414]


## Phrasal verbs: https://www.englishpage.com/prepositions/phrasaldictionary.html

In [18]:
import requests
from bs4 import BeautifulSoup

def extraer_ejemplos_phrasal_verbs(url):
    try:
        # Realizar la solicitud para obtener el contenido de la página
        response = requests.get(url)
        response.raise_for_status()

        # Analizar el contenido HTML de la página
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extraer solo las frases de la columna "Example"
        ejemplos = []
        for row in soup.find_all("tr")[1:]:
            cells = row.find_all("td")
            if len(cells) == 3:
                # crear una tripleta (verbo, significado, ejemplo) para los ejemplos substituir "\n" por " "
                ejemplos.append((cells[0].get_text(strip=True), cells[1].get_text(strip=True), cells[2].get_text(strip=True).replace("\n", " ")))
        return ejemplos
    except requests.RequestException as e:
        print(f"Error al acceder a la página: {e}")
        return []

# URL de la página con ejemplos de phrasal verbs
# url = "https://www.englishpage.com/prepositions/phrasaldictionaryA.html"

ejemplos_phrasal_verbs = []

# Iterar a través de cada letra del alfabeto
for letra in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    url = f"https://www.englishpage.com/prepositions/phrasaldictionary{letra}.html"       
    # Llamar a la función y obtener los resultados
    ejemplos_phrasal_verbs.extend(extraer_ejemplos_phrasal_verbs(url))

# Mostrar los primeros 5 ejemplos
print(ejemplos_phrasal_verbs[:20])


[('act up', 'behave or function improperly', "I think I need to take my car to the mechanic because it's acting up again."), ('add * up +', 'calculate a sum', 'I added up the receipts and it totaled $135.46.'), ('add up to +', 'equal an amount', 'The total expenses added up to $325.00.'), ('add up', 'make sense', "Her story doesn't add up. I think she is lying."), ('ask * out +', 'invite on a date', "I can't believe that Joe finally asked me out on a date!")]


In [19]:
len(ejemplos_phrasal_verbs)

549

In [20]:
import csv

# Define the file path
csv_file_path = './datasets/phrasal_verbs.csv'

# Create the CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file, delimiter='~')
    writer.writerow(['verb', 'meaning', 'example'])

    # Write the data to the CSV file
    for verb, meaning, example in ejemplos_phrasal_verbs:
        writer.writerow([verb, meaning, example])

print(f"CSV file created: {csv_file_path}")


CSV file created: ./datasets/phrasal_verbs.csv


In [2]:
import pandas as pd

# Leer los primeros 10 registros del archivo CSV
csv_file_path = './datasets/phrasal_verbs.csv'
csv_data = pd.read_csv(csv_file_path, sep='~') #, nrows=10)
csv_data.head()

,verb,meaning,example
0,act up,behave or function improperly,I think I need to take my car to the mechanic ...
1,add * up +,calculate a sum,I added up the receipts and it totaled $135.46.
2,add up to +,equal an amount,The total expenses added up to $325.00.
3,add up,make sense,Her story doesn't add up. I think she is lying.
4,ask * out +,invite on a date,I can't believe that Joe finally asked me out ...


In [6]:
# show all rows with any column with empty values
csv_data[csv_data.isnull().any(axis=1)]

,verb,meaning,example


In [12]:
# longitud del dataset
print(csv_data.shape)

# imprimir todos los registros
# pd.set_option('display.max_rows', 100) #csv_data.shape[0]+1)

# mostrar los registros del 20 al 30
csv_data[20:50]
# csv_data

(549, 3)


,verb,meaning,example
20,blow * up +,make explode;destroy using explosives,The terrorists blew the bridge up.
21,blow up,explode,The bomb blew up before they could defuse it.
22,blow up,suddenly become very angry,"When Joan heard the news, she blew up and rush..."
23,break * down +,analyze in detail,We need to break this problem down in order to...
24,break down,stop working properly,The truck broke down in the desert.
25,break down,become mentally ill,She broke down after her husband died.
26,break * in +,wear or use something new until it is\ncomfort...,I need to break these shoes in before I go hik...
27,break in,interrupt,"While we were discussing the situation, Terri ..."
28,break in,enter a place unlawfully,The burglar broke in between midnight and 3 AM.
29,break in on +,interrupt (a conversation),Jane broke in on the conversation and told us ...


### A partir de ejemplos de phrasal verbas generamos con gpt4 sintéticamente ejemplos con traducciones

PROMPT:

quiero que hagas un script en python que a partir de los siguientes ejemplos de phrasal verbs crees 3 ejemplos similares de cada uno de ellos junto con las traducciones y que los metas en un fichero .csv de 2 columnas (spanish~english)  cuyo caracter de separación es "~" 
ejemplos:
Cut out – Dejar de hacer algo.
My Dad has very high cholesterol, so he has cut out eggs and dairy from his diet.
Cut back – Hacer algo en menor medida.
You eat too much salt, you should cut back on salty foods.
Cut off – Recortar.
He cut all the fat off.
Cut out – Dar forma a algo mediante cortes.
He cut several pieces of pastry out.
Cut up – Recortar en piezas con un cuchillo.
He cut the pie up into equal slices.


Ahora crea la lista de pares de esta manera:
ejemplos = [(ejemplo1 que usa "Get on" en español, traducción en inglés), (ejemplo2 que usa "Get on" en español, traducción en inglés),  (ejemplo3 que usa "Get on" en español, traducción en inglés), (ejemplo1 que usa "Get on-part2" en español, traducción en inglés), (ejemplo2 que usa "Get on-part2" en español, traducción en inglés), (ejemplo3 que usa "Get on-part2" en español, traducción en inglés), ...]

#### Script compuesto a base de concatenar resultados del prompt anterior para generar ejemplos de phrasal verbs con traducciones.

In [6]:
# Crear ejemplos similares para cada phrasal verb y sus traducciones al español

ejemplos = [
    # Get on
    ("Se subieron al autobús", "They got on the bus"),
    ("Vamos a subirnos al avión", "Let's get on the plane"),
    ("Él se subió a su bicicleta y se fue", "He got on his bike and left"),
    
    # Get on (part 2)
    ("Pedro se está adaptando muy bien en su nuevo hogar", "Pedro is getting on very well in his new home"),
    ("Ella se lleva muy bien con sus compañeros", "She is getting on very well with her colleagues"),
    ("Estoy progresando bastante en mi proyecto", "I'm getting on quite well with my project"),

    # Get on for
    ("Ella debe estar rondando los cincuenta años", "She must be getting on for fifty"),
    ("Creo que está cerca de los treinta", "I think he's getting on for thirty"),
    ("Deben estar rondando los cuarenta", "They must be getting on for forty"),

    # Get across
    ("Me cuesta hacer entender mi punto de vista", "I struggle to get my point across"),
    ("Intentó transmitir sus emociones, pero fue difícil", "He tried to get his emotions across, but it was hard"),
    ("Quiero asegurarme de que mi mensaje se entienda claramente", "I want to make sure I'm getting my message across clearly"),

    # Get at
    ("¿A qué te refieres con eso?", "What are you getting at with that?"),
    ("No entiendo lo que intentas insinuar", "I don't understand what you're getting at"),
    ("Explícate mejor, no sé a qué quieres llegar", "Explain yourself better, I don’t know what you're getting at"),

    # Get down to
    ("Debemos empezar a concentrarnos en el proyecto", "We must get down to the project"),
    ("Es hora de que te pongas serio con tus estudios", "It's time you got down to your studies"),
    ("Vamos a dedicarnos al trabajo de verdad", "Let's get down to some real work"),

    # Get off with
    ("Se libraron con solo una advertencia", "They got off with just a warning"),
    ("Consiguieron salir del problema con una multa mínima", "They managed to get off with a minimal fine"),
    ("Ella se salvó de un castigo mayor", "She got off with a lesser punishment"),

    # Get over
    ("No me puedo creer lo rápido que se recuperó", "I can't get over how quickly he recovered"),
    ("Es asombroso lo mucho que ha crecido", "It’s amazing how much she has grown"),
    ("No puedo superar lo hermoso que es este lugar", "I can’t get over how beautiful this place is"),

    # Get over with
    ("Quiero terminar esto lo antes posible", "I want to get this over with as soon as possible"),
    ("No puedo esperar a que termine esta reunión", "I can't wait to get this meeting over with"),
    ("Será mejor que hagamos esto rápido", "We better get this done with quickly"),

    # Get round to
    ("Todavía no he tenido tiempo de leer ese libro", "I haven’t got round to reading that book yet"),
    ("Algún día arreglaré ese reloj", "Someday I'll get round to fixing that clock"),
    ("No he tenido la oportunidad de llamarla aún", "I haven’t got round to calling her yet"),

    # Get up to
    ("Los niños están tramando algo en su habitación", "The children are getting up to something in their room"),
    ("Me pregunto qué estarán haciendo ahora", "I wonder what they are getting up to now"),
    ("Seguramente están haciendo travesuras", "They must be getting up to mischief"),
    
    # Take in
    ("No te dejes engañar por su apariencia", "Don’t be taken in by his appearance"),
    ("Se tragó toda la historia sin dudar", "He took in the whole story without doubting"),
    ("Ella se dejó engañar por las promesas", "She was taken in by the promises"),

    # Take (it) out on
    ("No desquites tu frustración con los niños", "Don’t take your frustration out on the kids"),
    ("Él a menudo descarga su enojo en sus amigos", "He often takes his anger out on his friends"),
    ("No deberías desquitarte conmigo", "You shouldn’t take it out on me"),

    # Take off (imitate)
    ("Ella imita a la profesora perfectamente", "She takes off the teacher perfectly"),
    ("Juan hace una buena imitación de su jefe", "Juan takes off his boss really well"),
    ("Ellos hacen parodias de celebridades", "They take off celebrities brilliantly"),

    # Take on (new lease of life)
    ("Mi abuelo está mucho más activo desde su jubilación", "My grandfather has taken on a new lease of life since his retirement"),
    ("Después de mudarse, ella parece rejuvenecida", "After moving, she has taken on a new lease of life"),
    ("Tras superar su enfermedad, él está lleno de energía", "Since overcoming his illness, he has taken on a new lease of life"),

    # Take on (too much)
    ("Él se ha comprometido con demasiados proyectos", "He has taken on too many projects"),
    ("Están asumiendo más responsabilidades de las que pueden manejar", "They are taking on more responsibilities than they can handle"),
    ("Ella intenta hacer demasiadas cosas a la vez", "She is trying to take on too much at once"),

    # Take out (insurance)
    ("Él sacó un seguro para su coche nuevo", "He took out insurance for his new car"),
    ("Decidieron asegurar su casa contra incendios", "They decided to take out fire insurance for their house"),
    ("Tomó un seguro de salud para su familia", "He took out health insurance for his family"),

    # Take over
    ("La corporación se apoderó de la pequeña empresa", "The corporation tried to take over the small business"),
    ("El nuevo gerente se hizo cargo de la oficina", "The new manager took over the office"),
    ("Los inversores intentaron tomar el control de la compañía", "The investors attempted to take over the company"),

    # Take to someone
    ("Rápidamente se encariñó con su nuevo equipo", "He quickly took to his new team"),
    ("Ella se apegó a su nueva mascota inmediatamente", "She took to her new pet immediately"),
    ("Se adaptó a su nuevo entorno sorprendentemente rápido", "He took to his new environment surprisingly fast"),

    # Take away
    ("Ella se llevó los documentos de la mesa", "She took the documents away from the table"),
    ("Se llevaron los muebles viejos", "They took the old furniture away"),
    ("Tomó su teléfono y se fue", "He took his phone away and left"),

    # Take up
    ("La construcción ocupó mucho espacio", "The construction took up a lot of space"),
    ("El nuevo proyecto consume todo mi tiempo", "The new project takes up all my time"),
    ("Este hobby está ocupando mucho de su tiempo libre", "This hobby is taking up much of her free time"),

    # Look up (Buscar una referencia)
    ("Busca la palabra en el diccionario", "Look up the word in the dictionary"),
    ("Voy a buscar la dirección en Internet", "I'm going to look up the address on the Internet"),
    ("Ella buscó la receta en un libro de cocina", "She looked up the recipe in a cookbook"),

    # Look someone up (Visitar)
    ("Si alguna vez visitas Madrid, ven a verme", "If you ever visit Madrid, look me up"),
    ("Cuando esté en Nueva York, buscaré a un viejo amigo", "When I'm in New York, I'll look up an old friend"),
    ("Deberías buscar a tus primos cuando vayas a París", "You should look up your cousins when you go to Paris"),

    # Look after (Cuidar de)
    ("Voy a cuidar del perro de mi hermano este fin de semana", "I'm going to look after my brother's dog this weekend"),
    ("Ella cuida de sus plantas como si fueran sus hijos", "She looks after her plants as if they were her children"),
    ("Nosotros cuidaremos de la casa mientras estás fuera", "We will look after the house while you're away"),

    # Look into (Investigar)
    ("El detective investigará las huellas en la escena", "The detective will look into the fingerprints at the scene"),
    ("Debemos investigar todas las posibilidades", "We should look into all possibilities"),
    ("Ella dijo que investigaría el asunto", "She said she would look into the matter"),

    # Look on (Considerar)
    ("Consideramos este lugar como un santuario", "We look on this place as a sanctuary"),
    ("Ellos ven su casa como su lugar de paz", "They look on their home as their place of peace"),
    ("Ella siempre ha visto la ciudad como su hogar", "She has always looked on the city as her home"),

    # Go after
    ("Ella está persiguiendo su sueño de ser artista", "She is going after her dream of being an artist"),
    ("Él va tras el ladrón", "He is going after the thief"),
    ("Estamos persiguiendo un objetivo común", "We are going after a common goal"),

    # Go back on
    ("Él rompió su promesa de llegar temprano", "He went back on his promise to arrive early"),
    ("La compañía no cumplió con su acuerdo", "The company went back on its agreement"),
    ("No puedo creer que hayas faltado a tu palabra", "I can't believe you went back on your word"),

    # Go in for (habit)
    ("No me interesa ese tipo de actividades", "I don’t go in for that kind of activities"),
    ("Ella no se inclina por los deportes extremos", "She doesn’t go in for extreme sports"),
    ("No suelo interesarme por la política", "I don't usually go in for politics"),

    # Go in for (competition)
    ("¿Vas a participar en la competencia de natación?", "Are you going in for the swimming competition?"),
    ("Él decidió competir en el torneo", "He decided to go in for the tournament"),
    ("Estamos considerando entrar en el concurso", "We are thinking of going in for the contest"),

    # Go off (food)
    ("Estos tomates ya se pusieron malos", "These tomatoes have gone off"),
    ("El pescado huele mal, debe haberse echado a perder", "The fish smells bad, it must have gone off"),
    ("Creo que esta fruta se descompuso", "I think this fruit has gone off"),

    # Go on
    ("Algo extraño está pasando aquí", "Something strange is going on here"),
    ("¿Qué sucede en la calle?", "What is going on in the street?"),
    ("Este evento sigue ocurriendo cada año", "This event keeps going on every year"),

    # Go round
    ("No hay suficientes sillas para todos", "There aren’t enough chairs to go round"),
    ("La comida no fue suficiente para todos los invitados", "The food wasn’t enough to go round for all the guests"),
    ("Nos aseguraremos de que haya suficientes recursos para todos", "We will make sure there is enough to go round for everyone"),

    # Go through with
    ("No pudo completar su plan de mudarse al extranjero", "He couldn’t go through with his plan to move abroad"),
    ("Cuando llegó el momento, ella no pudo casarse", "When the time came, she couldn’t go through with the wedding"),
    ("Finalmente, decidieron no seguir adelante con el negocio", "In the end, they decided not to go through with the deal"),

    # Come after
    ("Un gato me siguió cuando empecé a correr", "A cat came after me when I started running"),
    ("Ellos vinieron tras nosotros después del concierto", "They came after us after the concert"),
    ("El policía vino tras el ladrón", "The policeman came after the thief"),

    # Come about
    ("Déjame explicar cómo surgió la idea", "Let me explain how the idea came about"),
    ("¿Cómo ocurrió el accidente?", "How did the accident come about?"),
    ("Nadie sabe cómo se originó el problema", "No one knows how the problem came about"),

    # Come down to
    ("Todo se reduce a si realmente quieres el trabajo", "It all comes down to whether you really want the job"),
    ("Al final, se resume en la calidad, no la cantidad", "In the end, it comes down to quality, not quantity"),
    ("La decisión se reduce a lo que el comité piense", "The decision comes down to what the committee thinks"),

    # Come in for
    ("El actor recibió muchas críticas por su actuación", "The actor has come in for a lot of criticism over his performance"),
    ("La nueva política ha recibido mucha culpa", "The new policy has come in for a lot of blame"),
    ("El director se llevó todas las críticas por el fracaso", "The director has come in for all the criticism over the failure"),

    # Come off
    ("El evento salió bien al final", "The event came off well in the end"),
    ("La fiesta no tuvo éxito después de todo", "The party didn't come off after all"),
    ("El plan no se realizó como esperábamos", "The plan didn't come off as we hoped"),

    # Come out
    ("Las estrellas salieron en la noche", "The stars have come out in the night"),
    ("Todas las rosas florecieron", "All the roses have come out"),
    ("El sol salió después de la tormenta", "The sun came out after the storm"),

    # Come up
    ("Mira, ha surgido un imprevisto y no puedo ir", "Look, something has come up and I can't go"),
    ("Un problema inesperado ha surgido", "An unexpected problem has come up"),
    ("Ha surgido un conflicto y necesitamos resolverlo", "A conflict has come up and we need to solve it"),

    # Come up against
    ("Nos encontramos con una dificultad en el proyecto", "We've come up against a difficulty in the project"),
    ("El equipo se encontró con un desafío grande", "The team has come up against a big challenge"),
    ("Ella se encontró con una barrera al aprender el idioma", "She came up against a barrier when learning the language"),

    # Come up to
    ("La película no cumplió con las expectativas", "The movie didn't come up to expectations"),
    ("El resultado no estuvo a la altura de lo esperado", "The result didn't come up to what was expected"),
    ("Su desempeño no alcanzó el nivel necesario", "His performance didn’t come up to the necessary level"),

    # Come up with
    ("Todavía tenemos que pensar en una estrategia", "We still have to come up with a strategy"),
    ("Necesitamos idear un mejor plan", "We need to come up with a better plan"),
    ("Han ideado una solución creativa", "They have come up with a creative solution"),

    # Put across
    ("Él no pudo transmitir su mensaje correctamente", "He couldn’t put his message across correctly"),
    ("Ella no pudo expresar sus emociones", "She couldn’t put her emotions across"),
    ("El profesor no pudo explicar la lección", "The teacher couldn’t put the lesson across"),

    # Put someone down
    ("Ella siempre me menosprecia", "She always puts me down"),

    # Turn down – Rechazar
    ("Rechacé la oferta porque no me interesaba", "I turned down the offer because it didn't interest me"),
    ("Ella rechazó la invitación a la fiesta", "She turned down the invitation to the party"),
    ("Ellos rechazaron la propuesta del proyecto", "They turned down the project proposal"),

    # Turn out – Algo que resulta al final
    ("Resultó ser un buen día después de todo", "It turned out to be a good day after all"),
    ("La reunión resultó mejor de lo esperado", "The meeting turned out better than expected"),
    ("El experimento resultó ser un éxito", "The experiment turned out to be a success"),

    # Turn out – Reunirse o formar un grupo
    ("Mucha gente se reunió para el concierto", "Many people turned out for the concert"),
    ("Una gran multitud se formó para la protesta", "A large crowd turned out for the protest"),
    ("Los estudiantes se reunieron para la asamblea", "The students turned out for the assembly"),

    # Turn up – Descubrir por causalidad
    ("No te preocupes, seguro que tus llaves aparecerán", "Don't worry, your keys are bound to turn up"),
    ("Mi viejo reloj apareció mientras limpiaba", "My old watch turned up while I was cleaning"),
    ("Finalmente, el documento perdido apareció", "Finally, the lost document turned up"),

    # Turn up – Llegar o aparecer
    ("Pocas personas asistieron a la reunión", "Not many people turned up for the meeting"),
    ("Él apareció inesperadamente en la fiesta", "He turned up unexpectedly at the party"),
    ("Solo unos pocos asistieron al evento", "Only a few turned up for the event"),

    # Put away – Guardar algo en su lugar correcto
    ("Pon tus juguetes en su lugar", "Put your toys away"),
    ("Deberías guardar tus cosas en el armario", "You should put your things away in the closet"),
    ("No dejes tus cosas tiradas por ahí", "Don’t leave your things lying around"),

    # Make after – Perseguir
    ("Ella corrió tras el ladrón pero no pudo alcanzarlo", "She made after the thief but could not catch him"),
    ("El perro fue tras el gato rápidamente", "The dog made after the cat quickly"),
    ("Corrieron tras el autobús que se iba", "They made after the departing bus"),

    # Make for – Resultar o facilitar
    ("Este sistema facilita el proceso de aprendizaje", "This system makes for an easier learning process"),
    ("Una buena planificación resulta en un mejor resultado", "Good planning makes for a better outcome"),
    ("Trabajar en equipo facilita completar el proyecto", "Working together makes for completing the project easily"),

    # Make off with – Escapar
    ("El ladrón escapó con el dinero", "The thief made off with the money"),
    ("Escaparon con las joyas sin ser detectados", "They made off with the jewels undetected"),
    ("El gato se escapó con el pescado de la mesa", "The cat made off with the fish from the table"),

    # Make out – Pretender
    ("Juan pretendió no haber visto el semáforo", "Juan made out that he hadn’t seen the traffic light"),
    ("Ella fingió no conocer la verdad", "She made out that she didn’t know the truth"),
    ("Pretendieron estar sorprendidos por la noticia", "They made out to be surprised by the news"),

    # Make out – Ver o entender
    ("No pude entender lo que decía la carta", "I couldn’t quite make out what the letter said"),
    ("Ella intentó leer el cartel a lo lejos", "She tried to make out the sign in the distance"),
    ("No entendíamos lo que el profesor explicaba", "We couldn’t make out what the teacher was explaining"),

    # Make someone out – Comprender la conducta de alguien
    ("No entiendo su comportamiento en la reunión", "I can’t make him out in the meeting"),
    ("Es difícil comprender sus motivos", "It's hard to make her motives out"),
    ("No puedo entender por qué actuó así", "I can’t make out why he acted that way"),

    # Make up – Inventar
    ("Creo que inventó una excusa para no venir", "I think he made up an excuse not to come"),
    ("Ella se inventó una historia increíble", "She made up an incredible story"),
    ("Inventaron un juego para pasar el tiempo", "They made up a game to pass the time"),

    # Make up for – Compensar por algo
    ("El regalo compensó su ausencia en mi cumpleaños", "The gift made up for his absence on my birthday"),
    ("Estos momentos felices compensan los tiempos difíciles", "These happy moments make up for the tough times"),
    ("Su éxito compensó todos los sacrificios hechos", "Her success made up for all the sacrifices made"),

    # Put back – Devolver algo a su lugar
    ("Pon el libro de vuelta en la estantería", "Put the book back on the shelf"),
    ("Deberías devolver el libro a la biblioteca", "You should put the book back in the library"),
    ("No dejes tus cosas tiradas por ahí", "Don’t leave your things lying around"),

    # Set about
    ("Debemos comenzar a planificar las vacaciones.", "We must set about planning the vacation."),
    ("Ella se puso a limpiar la casa.", "She set about cleaning the house."),
    ("Es hora de que comiencen a trabajar en el proyecto.", "It's time they set about working on the project."),
    
    # Set in
    ("Parece que el frío se ha establecido para el invierno.", "It seems the cold has set in for the winter."),
    ("La oscuridad se estableció rápidamente.", "The darkness set in quickly."),
    ("Una vez que la desesperación se establece, es difícil salir de ella.", "Once despair sets in, it's hard to get out of it."),
    
    # Set out – Dar detalles escritos
    ("Este informe detalla todos los gastos.", "This report sets out all the expenses."),
    ("La ley establece los derechos y obligaciones.", "The law sets out the rights and obligations."),
    ("El contrato detalla las condiciones del acuerdo.", "The contract sets out the terms of the agreement."),
    
    # Set out – Preparar
    ("He dispuesto todo para la reunión.", "I’ve set out everything for the meeting."),
    ("Los libros fueron dispuestos en la mesa.", "The books were set out on the table."),
    ("Organizó los ingredientes para cocinar.", "She set out the ingredients for cooking."),
    
    # Set out – Comenzar algo
    ("Ella se propuso aprender un nuevo idioma.", "She set out to learn a new language."),
    ("Se propuso correr un maratón este año.", "He set out to run a marathon this year."),
    ("Se propusieron cambiar la industria con su nueva tecnología.", "They set out to change the industry with their new technology."),
    
    # Set up – Establecer
    ("El gobierno estableció un nuevo comité.", "The government set up a new committee."),
    ("Configuraron un nuevo sistema de seguridad.", "They set up a new security system."),
    ("Establecieron una tienda en el centro.", "They set up a shop downtown."),
    
    # Set (up) on – Atacar
    ("Fueron atacados por un grupo de perros callejeros.", "They were set upon by a group of stray dogs."),
    ("El equipo fue emboscado por los rivales.", "The team was set upon by the rivals."),
    ("Fue atacada por la crítica.", "She was set upon by the critics."),

    # Put forward – Proponer, presentar
    ("Ellos propusieron un nuevo plan.", "They put forward a new plan."),
    ("El gobierno presentó un nuevo proyecto de ley.", "The government put forward a new bill."),
    ("El equipo propuso una nueva estrategia.", "The team put forward a new strategy."),
    
    # Give away - Delatar, Traicionar
    ("Sus mentiras lo delataron", "His lies gave him away"),
    ("Su expresión facial la delató", "Her facial expression gave her away"),
    ("El tono de su voz lo delató", "The tone of his voice gave him away"),

    # Give off - Emitir mal olor
    ("La basura empezó a emitir un olor extraño", "The garbage began to give off a strange smell"),
    ("Esa planta emite un olor muy peculiar", "That plant gives off a very peculiar smell"),
    ("El tubo de escape del coche emitía humo", "The car's exhaust pipe was giving off smoke"),

    # Give out - Terminarse
    ("La comida se terminó durante la fiesta", "The food gave out during the party"),
    ("Nuestra paciencia se agotó", "Our patience gave out"),
    ("La energía de la batería se acabó", "The battery's energy gave out"),

    # Give over - Abandonarse a una actividad
    ("El fin de semana lo dedicamos a ver películas", "The weekend was given over to watching movies"),
    ("Se entregó por completo a la pintura", "He gave himself over to painting"),
    ("La tarde se dedicó a la lectura de novelas", "The afternoon was given over to reading novels"),

    # Give over - Parar
    ("¿Por qué no paras? Me estás molestando", "Why don't you give over? You are bothering me"),
    ("Deja de quejarte todo el tiempo", "Give over complaining all the time"),
    ("Para de hablar y escucha", "Give over talking and listen"),

    # Give up - Rendirse
    ("El equipo se rindió después de un largo partido", "The team gave up after a long match"),
    ("Finalmente se rindió en su intento de resolver el problema", "She finally gave up trying to solve the problem"),
    ("No te rindas en tus sueños", "Don't give up on your dreams"),

    # Give up - Dar algo o alguien por muerto o perdido
    ("Después de horas de búsqueda, dieron al gato por perdido", "After hours of searching, the cat was given up for lost"),
    ("La policía dio por muerto al desaparecido", "The police gave the missing person up for dead"),
    ("Dieron por terminado el proyecto", "The project was given up for over"),

    # Give up - Dejar una actividad o hábito
    ("Dejó de fumar hace un año", "He gave up smoking a year ago"),
    ("Ella decidió dejar de beber café", "She gave up drinking coffee"),
    ("Abandonaron la idea de irse de vacaciones", "They gave up the idea of going on vacation"),

    # Cut out – Dejar de hacer algo
    ("Mi hermana ha dejado de fumar", "My sister has cut out smoking"),
    ("Dejé de comer comida rápida para mejorar mi salud", "I have cut out fast food to improve my health"),
    ("Él decidió dejar de beber café", "He decided to cut out coffee"),

    # Cut back – Hacer algo en menor medida
    ("Deberías reducir el consumo de azúcar", "You should cut back on sugar"),
    ("Voy a reducir las horas que paso frente al televisor", "I'm going to cut back on the hours I spend watching TV"),
    ("Ella redujo el uso del teléfono móvil", "She cut back on using her mobile phone"),

    # Cut off – Recortar
    ("Él recortó todas las partes quemadas del pastel", "He cut off all the burnt parts of the cake"),
    ("Recortaron la tela sobrante", "They cut off the excess fabric"),
    ("Cortó las puntas dañadas de su cabello", "She cut off the damaged ends of her hair"),

    # Cut out – Dar forma a algo mediante cortes
    ("Cortó figuras de papel para la decoración", "He cut out paper shapes for the decoration"),
    ("Ella cortó un corazón de la tela", "She cut a heart out of the fabric"),
    ("Recortaron siluetas de animales para el proyecto escolar", "They cut out animal silhouettes for the school project"),

    # Cut up – Recortar en piezas con un cuchillo
    ("Cortó la zanahoria en rodajas para la ensalada", "He cut up the carrot into slices for the salad"),
    ("Ella cortó la pizza en ocho partes", "She cut up the pizza into eight pieces"),
    ("Cortaron la sandía en trozos para la fiesta", "They cut up the watermelon into pieces for the party"),

    # Put in – Instalar, poner
    ("Instalamos un nuevo sistema de seguridad", "We put in a new security system"),
    ("Ellos pusieron un nuevo sistema de calefacción", "They put in a new heating system"),
    ("Instalamos un nuevo sistema de riego", "We put in a new irrigation system"),

    # Put off – Aplazar
    ("Decidieron aplazar la reunión", "They decided to put off the meeting"),
    ("Ellos pospusieron la fiesta para el próximo fin de semana", "They put off the party until next weekend"),
    ("Ellos decidieron aplazar el viaje", "They decided to put off the trip"),

    # Put on – Ponerse ropa
    ("Él se puso su traje para la fiesta", "He put on his suit for the party"),
    ("Ella se puso su vestido favorito", "She put on her favorite dress"),
    ("Ellos se pusieron sus mejores ropas", "They put on their best clothes"),
    ("Ella se puso maquillaje", "She put on makeup"),

    # Bring about
    ("La guerra fue causada por conflictos políticos", "The war was brought about by political conflicts"),
    ("Su decisión causó muchos cambios en la empresa", "His decision brought about many changes in the company"),
    ("Ese evento provocó una nueva era en la tecnología", "That event brought about a new era in technology"),

    # Bring off
    ("Lograron terminar el proyecto a tiempo contra todo pronóstico", "They finally brought the project off on time against all odds"),
    ("Ella consiguió un gran logro en su carrera profesional", "She brought off a great achievement in her professional career"),
    ("El mago realizó el truco con éxito", "The magician brought the trick off successfully"),

    # Bring on (disease)
    ("El frío extremo le provocó una gripe", "The extreme cold brought on a flu"),
    ("Estar en la lluvia le causó un resfriado", "Being in the rain brought on a cold"),
    ("La falta de sueño le provocó dolores de cabeza", "Lack of sleep brought on headaches"),

    # Bring on (problem)
    ("Sus acciones le causaron problemas con la ley", "His actions brought these problems on himself"),
    ("Ignorar el consejo le trajo dificultades", "Ignoring the advice brought on difficulties"),
    ("Su comportamiento irresponsable le causó muchos inconvenientes", "His irresponsible behavior brought on many troubles"),

    # Bring round (influence)
    ("Finalmente convenció a su familia para que apoyara su carrera artística", "He finally brought his family round to support his artistic career"),
    ("Después de mucho debatir, ella convenció a su equipo para que aceptara el plan", "After much debate, she brought her team round to the plan"),
    ("Consiguió cambiar la opinión de su jefe sobre el proyecto", "He managed to bring his boss round to the project"),

    # Bring up (mention)
    ("Ella mencionó un punto interesante durante la reunión", "She brought up an interesting point during the meeting"),
    ("Durante la cena, trajeron a colación el tema del viaje", "During dinner, they brought up the topic of the trip"),
    ("No olvides mencionar el problema en la próxima junta", "Don't forget to bring up the issue at the next meeting"),

    # Settle down
    ("Después de viajar por el mundo, se estableció en su ciudad natal", "After traveling around the world, he settled down in his hometown"),
    ("Después de muchos años de aventuras, decidió asentarse", "After many years of adventures, he decided to settle down"),
    ("Después de la jubilación, se establecieron en el campo", "After retirement, they settled down in the countryside"),

    # Call after
    ("El bebé tiene el nombre de su abuelo", "The baby is named after his grandfather"),
    ("Ella fue llamada como su bisabuela", "She was called after her great-grandmother"),
    ("Juan fue nombrado en honor a su tío", "Juan was called after his uncle"),

    # Call at
    ("El autobús hace una parada en la ciudad", "The bus calls at the city"),
    ("El tren hará una parada en la próxima estación", "The train will call at the next station"),
    ("Haremos una parada breve en el pueblo", "We will call at the town for a short stop"),

    # Call away
    ("La enfermera tuvo que irse por una emergencia", "The nurse was called away for an emergency"),
    ("El bombero fue llamado para un rescate", "The firefighter was called away for a rescue"),
    ("El policía fue requerido de urgencia", "The police officer was called away urgently"),

    # Call back
    ("Te devolveré la llamada en una hora", "I'll call you back in an hour"),
    ("Ella dijo que me llamaría de vuelta mañana", "She said she would call back tomorrow"),
    ("Por favor, devuélveme la llamada cuando puedas", "Please call me back when you can"),

    # Call for (pick up)
    ("Voy a pasar por ti a las 7", "I will call for you at 7"),
    ("Dijo que pasaría por mí después del trabajo", "He said he would call for me after work"),
    ("Ella vendrá a buscarme esta noche", "She will call for me tonight"),

    # Call for (demand)
    ("Los ciudadanos exigen una nueva ley", "The citizens called for a new law"),
    ("La situación requiere medidas inmediatas", "The situation calls for immediate measures"),
    ("La protesta pide cambios en el gobierno", "The protest calls for changes in the government"),

    # Call for (require)
    ("Este trabajo requiere paciencia", "This job calls for patience"),
    ("Ser chef exige creatividad", "Being a chef calls for creativity"),
    ("El papel demanda habilidades de actuación", "The role calls for acting skills"),

    # Call in (to work)
    ("Llamé al electricista para el problema de luz", "I called in the electrician for the lighting issue"),
    ("Necesitamos llamar a un experto", "We need to call in an expert"),
    ("Voy a llamar al técnico para la reparación", "I'm going to call in the technician for the repair"),

    # Call in (on) (visit)
    ("Pasé a visitar a mis abuelos", "I called in on my grandparents"),
    ("Vamos a visitar a nuestros amigos en su casa", "We'll call in on our friends at their home"),
    ("Él pasó a ver a su hermana en el hospital", "He called in on his sister in the hospital"),

    # Call off
    ("El evento fue cancelado debido al mal tiempo", "The event was called off due to bad weather"),
    ("Decidieron cancelar la reunión", "They decided to call off the meeting"),
    ("El concierto se suspendió por la lluvia", "The concert was called off because of the rain"),

    # Call on/upon
    ("Invito al director a hablar", "I now call on the director to speak"),
    ("Pedimos al comité que tome una decisión", "We call upon the committee to make a decision"),
    ("Se solicita al experto dar su opinión", "The expert is called on to give his opinion"),

    # Call out
    ("Él gritó mi nombre en la multitud", "He called out my name in the crowd"),
    ("El profesor mencionó a los estudiantes en voz alta", "The teacher called out the students loudly"),
    ("Ella exclamó la respuesta en la clase", "She called out the answer in class"),

    # Call round
    ("La vecina pasó a saludar", "The neighbor called round to say hello"),
    ("Voy a ir a ver a mi tío esta tarde", "I'm going to call round to see my uncle this afternoon"),
    ("Ella pasará a recogerme más tarde", "She will call round to pick me up later"),

    # Call up (phone) (solicitado, convocado)
    ("Llámame cuando llegues a casa", "Call me up when you get home"),
    ("Ella llamó a su novio para decirle que llegaría tarde", "She called up her boyfriend to tell him she would be late"),
    ("Voy a llamar a mi hermana para que me ayude", "I will call up my sister to help me"),
    ("Mi abuelo fue llamado a filas durante la guerra", "My grandfather was called up during the war"),

    # Pick up (Recoger algo o a alguien)
    ("Voy a recoger unos documentos de la oficina", "I will pick some documents up from the office"),
    ("Puedes recogerme después del trabajo?", "Can you pick me up after work?"),
    ("Recogió su abrigo del suelo", "She picked her coat up off the floor"),

    # Pick up (Mejorar)
    ("Parece que mi salud está mejorando", "It seems my health is picking up"),
    ("El negocio está mejorando después de un mal mes", "The business is picking up after a bad month"),
    ("El equipo está jugando mejor últimamente", "The team is picking up lately"),

    # Pick at (Comer despacio)
    ("Juan picotea su comida cuando no tiene hambre", "Juan picks at his food when he's not hungry"),
    ("Solo picoteó su cena, casi no comió nada", "She just picked at her dinner, hardly ate anything"),
    ("Los niños a veces picotean en lugar de comer", "Kids sometimes pick at food instead of eating"),

    # Pick at (Irritar o molestar)
    ("No deberías irritar esa herida", "You shouldn’t pick at that wound"),
    ("Él siempre molesta a su hermano menor", "He always picks at his younger brother"),
    ("Evita irritar tu piel seca", "Avoid picking at your dry skin"),

    # Pick off (Quitar)
    ("Quité las hojas secas de la planta", "I picked the dry leaves off the plant"),
    ("Ella quitó las migas de la mesa", "She picked the crumbs off the table"),
    ("Quitó las etiquetas de la ropa nueva", "He picked the tags off the new clothes"),

    # Pick on (Acosar)
    ("Los niños suelen acosar al más pequeño del grupo", "Kids often pick on the smallest one in the group"),
    ("No acoses a tu hermana todo el tiempo", "Don't pick on your sister all the time"),
    ("El jefe acosa a los nuevos empleados", "The boss always picks on new employees"),

    # Pick up (Aprender mediante experiencia)
    ("Aprendí algo de italiano en mi viaje", "I picked up a little Italian on my trip"),
    ("Ella aprendió francés viendo películas", "She picked up French from watching movies"),
    ("Recogí algunos consejos útiles en la conferencia", "I picked up some useful tips at the conference"),

    # Pick up (Acelerar o estar ocupado)
    ("El tráfico aumenta durante las horas pico", "Traffic really picks up during rush hour"),
    ("El ritmo de trabajo se acelera antes de las vacaciones", "Work pace picks up before the holidays"),
    ("El centro comercial se llena los fines de semana", "The mall really picks up on weekends"),

    # Pick up (Llevar a alguien en el coche)
    ("Siempre recoge a sus hijos de la escuela", "She always picks her kids up from school"),
    ("Puedes recogerme en la estación?", "Can you pick me up at the station?"),
    ("Él recoge a su amigo del aeropuerto", "He picks his friend up from the airport"),

    # Pick up (Pagar la cuenta de alguien)
    ("Mi tío pagó la cena", "My uncle picked up the dinner tab"),
    ("Ofrecí pagar la cuenta del café", "I offered to pick up the coffee bill"),
    ("Ella pagó la factura del hotel", "She picked up the hotel bill"),

    # Pick out (Elegir algo)
    ("Elige un regalo para tu hermano", "Pick out a gift for your brother"),
    ("Tengo que elegir un vestido para la boda", "I have to pick out a dress for the wedding"),
    ("Selecciona los ingredientes para la receta", "Pick out the ingredients for the recipe"),

    # Carry away
    ("Mi hija se dejó llevar por la música en el concierto", "My daughter got a little carried away by the music at the concert"),
    ("A veces me dejo llevar por la emoción al hablar de mis hobbies", "Sometimes I get carried away when talking about my hobbies"),
    ("Se dejó llevar por la alegría en la fiesta de graduación", "He got carried away by the joy at the graduation party"),
    
    # Carry off
    ("Juan ganó un premio en la competencia de matemáticas", "Juan carried off a prize in the math competition"),
    ("La compañía se llevó un premio por su innovación", "The company carried off an award for its innovation"),
    ("Laura se llevó el primer lugar en el torneo de ajedrez", "Laura carried off the first place in the chess tournament"),
    
    # Carry on
    ("Por favor, sigue con lo que estabas haciendo", "Please, carry on with what you were doing"),
    ("Vamos a continuar con el plan como estaba previsto", "Let's carry on with the plan as it was intended"),
    ("Ella siguió con su discurso a pesar de las interrupciones", "She carried on with her speech despite the interruptions"),

    # Carry out (Llevar a cabo)
    ("El equipo llevará a cabo la tarea mañana", "The team will carry out the task tomorrow"),
    ("Vamos a realizar una encuesta para obtener más datos", "We will carry out a survey to gather more data"),
    ("El científico llevó a cabo un experimento exitoso", "The scientist carried out a successful experiment"),

    # Carry out (Cumplir)
    ("Siempre cumplo mis compromisos", "I always carry out my commitments"),
    ("El gerente cumplió con su promesa de mejorar las condiciones de trabajo", "The manager carried out his promise to improve working conditions"),
    ("Nos aseguraremos de cumplir con los objetivos del proyecto", "We will make sure to carry out the project's objectives"),

    # Carry through
    ("El equipo llevó a cabo el proyecto con éxito", "The team carried through the project successfully"),
    ("Llevamos a cabo el plan a pesar de los obstáculos", "We carried through the plan despite the obstacles"),
    ("Ella llevó a cabo su idea hasta el final", "She carried through her idea to the end"),

    # Be after
    ("¿Qué busca él? ¿Está buscando una solución?", "What's he after? Is he looking for a solution?"),
    ("Ella siempre anda tras nuevos desafíos", "She's always after new challenges"),
    ("¿Qué persigue el gato? Parece interesado en algo", "What is the cat after? It seems interested in something"),

    # Be against
    ("Estoy en contra de cambiar los planes a último momento", "I must say I'm against changing plans at the last minute"),
    ("Ellos se oponen a la construcción del nuevo centro comercial", "They are against the construction of the new mall"),
    ("Mi padre siempre se ha opuesto a esa idea", "My father has always been against that idea"),

    # Be along
    ("María llegará pronto, está en camino", "Maria will be along soon. She's on her way"),
    ("El autobús debería llegar en cualquier momento", "The bus should be along any moment"),
    ("Se espera que el paquete llegue antes del mediodía", "The package is expected to be along before noon"),

    # Be away
    ("Juan estará fuera esta semana, fue a un congreso", "Juan is away this week. He has gone to a conference"),
    ("Estaré fuera por unos días, voy de vacaciones", "I will be away for a few days. I'm going on vacation"),
    ("El gerente está fuera hoy, fue a una reunión en otra ciudad", "The manager is away today. He has gone to a meeting in another city"),

    # Be down (deprimido)
    ("Ana está un poco deprimida hoy debido a las malas noticias", "Ana is a bit down today because of the bad news"),
    ("Después de escuchar eso, me sentí un poco decaído", "After hearing that, I felt a bit down"),
    ("Está triste por los resultados del examen", "He's down about the exam results"),

    # Be down (descender)
    ("Los precios del mercado estuvieron bajos hoy", "The market prices were down today"),
    ("Las acciones de la compañía han disminuido este mes", "The company's stocks have been down this month"),
    ("La demanda de petróleo bajó inesperadamente", "The demand for oil was down unexpectedly"),

    # Be down on
    ("¿Por qué María desaprueba tanto al nuevo empleado?", "Why is Maria so down on the new employee?"),
    ("El jefe parece no gustarle el último proyecto", "The boss seems to be down on the latest project"),
    ("Ella siempre critica las nuevas propuestas", "She is always down on new proposals"),

    # Be down with
    ("Mi amigo está enfermo con gripe esta semana", "My friend is down with the flu this week"),
    ("He estado enfermo con una infección desde ayer", "I've been down with an infection since yesterday"),
    ("Ella está en cama con fiebre alta", "She's down with a high fever"),

    # Be in for
    ("Creo que nos espera una tormenta esta noche", "I think we are in for a storm tonight"),
    ("Nos esperan tiempos difíciles por delante", "We are in for some tough times ahead"),
    ("Este invierno parece que tendremos mucha nieve", "It seems we are in for a lot of snow this winter"),

    # Be in on
    ("Todos en el equipo participan en el plan", "Everyone in the team is in on the plan"),
    ("Ella también está involucrada en el acuerdo", "She is in on the deal as well"),
    ("Él está al tanto de la situación", "He is in on the situation"),

    # Be into
    ("Él está interesado en la música, especialmente el jazz", "He's into music, especially jazz"),
    ("Ella está muy metida en la lectura de novelas", "She's really into reading novels"),
    ("Los niños están muy metidos en los videojuegos últimamente", "The kids are really into video games lately"),

    # Be off
    ("¡Me voy! Tengo un tren que tomar", "I'm off! I've got a train to catch"),
    ("Él se va al trabajo", "He's off to work"),
    ("Ella salió para la escuela", "She's off to school"),

    # Be on – Tomar medicación
    ("El paciente está tomando medicamentos para el dolor", "The patient is on pain medication"),
    ("Estaba tomando antibióticos para la infección", "She was on antibiotics for the infection"),
    ("Se recetó medicamentos para su condición", "He was put on medication for his condition"),

    # Be on about – Querer decir
    ("¿A qué te refieres con eso?", "What are you on about with that?"),
    ("No entiendo lo que intentas decir", "I don’t understand what you’re on about"),
    ("¿De qué estás hablando exactamente?", "What exactly are you on about?"),

    # Be onto – Hacer un descubrimiento
    ("Los científicos están cerca de un gran descubrimiento", "The scientists are onto a major discovery"),
    ("Creo que estamos cerca de la solución", "I think we are onto the solution"),
    ("El detective está tras una pista importante", "The detective is onto an important lead"),

    # Be out – Estar fuera por un periodo corto
    ("¿Dónde está María? Está fuera en una reunión", "Where's Maria? She's out in a meeting"),
    ("Él no está, salió a almorzar", "He's not here, he's out for lunch"),
    ("Estaré fuera por unos días", "I'll be out for a few days"),

    # Be out of – Quedarse sin algo
    ("Nos hemos quedado sin leche", "We're out of milk"),
    ("La impresora se quedó sin tinta", "The printer is out of ink"),
    ("Se acabaron las galletas", "We are out of cookies"),

    # Be out to – Intentar conseguir algo
    ("Ella está decidida a ganar la competencia", "She's out to win the competition"),
    ("Están decididos a romper el récord", "They're out to break the record"),
    ("Está tratando de impresionar a su jefe", "He's out to impress his boss"),

    # Be up – Estar despierto
    ("Ella estaba despierta toda la noche", "She was up all night"),
    ("Estuve despierto antes del amanecer", "I was up before dawn"),
    ("Los niños ya están despiertos", "The kids are already up"),

    # Be up – Incrementar
    ("Los precios del petróleo han subido", "Oil prices are up"),
    ("El alquiler ha aumentado este año", "The rent is up this year"),
    ("Las ventas han subido este trimestre", "Sales are up this quarter"),

    # Be up to – Tramar
    ("¿Qué travesuras están planeando esos niños?", "What mischief are those kids up to?"),
    ("Algo está tramando, lo puedo sentir", "He is up to something, I can feel it"),
    ("Siempre están tramando alguna broma", "They're always up to some prank"),

    # Be up to – Ser responsabilidad de alguien
    ("Depende de nosotros encontrar una solución", "It's up to us to find a solution"),
    ("Es decisión tuya si quieres ir o no", "It's up to you whether you want to go or not"),
    ("Es responsabilidad del equipo terminar a tiempo", "It's up to the team to finish on time"),

    # See through (Descubrir la verdad)
    ("Descubrí sus verdaderas intenciones inmediatamente", "I saw through his true intentions immediately"),
    ("Vio a través de la excusa barata de su hermano", "She saw through her brother's flimsy excuse"),
    ("Nos dimos cuenta de la verdad detrás de sus palabras", "We saw through the truth behind his words"),

    # See about (Organizar, considerar)
    ("Veré si podemos organizar la reunión para mañana", "I'll see about arranging the meeting for tomorrow"),
    ("Ella se encargará de los preparativos para el evento", "She will see about the preparations for the event"),
    ("Voy a averiguar si hay entradas disponibles", "I'll see about whether there are tickets available"),

    # See into (Acompañar al alguien dentro de un sitio)
    ("El anfitrión nos acompañó hasta el salón", "The host saw us into the living room"),
    ("Me guiaron hasta la sala de conferencias", "They saw me into the conference room"),
    ("El portero llevó al visitante hasta la oficina", "The doorman saw the visitor into the office"),

    # See off (Perseguir para ahuyentar)
    ("El perro ahuyentó al ladrón rápidamente", "The dog soon saw the thief off"),
    ("Los vecinos ahuyentaron a los intrusos", "The neighbors saw the intruders off"),
    ("La seguridad desalojó a los manifestantes", "Security quickly saw the protesters off"),

    # See off (Ir a un transporte para decir adiós)
    ("Fui al aeropuerto para despedirme de ellos", "I went to the airport to see them off"),
    ("Nos reunimos en la estación para decir adiós", "We gathered at the station to see her off"),
    ("La familia se despidió de él en el puerto", "The family went to the port to see him off"),

    # See out (Acompañar a alguien a la salida)
    ("Te acompañaré a la puerta", "I'll see you out to the door"),
    ("Nos despidieron en la entrada", "They saw us out at the entrance"),
    ("El anfitrión nos acompañó hasta la salida", "The host saw us out to the exit"),

    # See through (Continuar hasta el final)
    ("Continuaron con el proyecto a pesar de los problemas", "They saw the project through despite the problems"),
    ("Ella siguió adelante con su plan hasta el final", "She saw her plan through to the end"),
    ("El equipo completó la tarea exitosamente", "The team saw the task through successfully"),

    # See through (Descubrir que alguien miente)
    ("Descubrieron rápidamente la mentira del ladrón", "They quickly saw through the thief's lie"),
    ("Ella vio a través del engaño", "She saw through the deception"),
    ("La policía descubrió el disfraz del criminal", "The police saw through the criminal's disguise"),

    # See to (Encargarse de algo)
    ("Se ocupó de los detalles y todo salió bien", "He saw to the details and everything went smoothly"),
    ("Ella se encargó de la organización del evento", "She saw to the organization of the event"),
    ("Nos aseguraremos de que todo esté listo", "We will see to it that everything is ready"),

    # Keep up
    ("¡Sigue así! Mantén el buen trabajo", "Well done! Keep up the good work"),
    ("Sigue estudiando duro para tus exámenes", "Keep up studying hard for your exams"),
    ("Debes seguir haciendo ejercicio regularmente", "You should keep up doing regular exercise"),

    # Keep around
    ("Siguen guardando esos viejos muebles por alguna razón", "They keep around those old furniture for some reason"),
    ("Ella siempre mantiene cerca su vieja cámara", "She always keeps her old camera around"),
    ("No sé por qué mantengo estas revistas viejas", "I have no idea why I keep these old magazines around"),

    # Keep at
    ("Carlos siguió con el proyecto hasta terminarlo", "Carlos kept at the project until he finished it"),
    ("Ella siguió practicando piano todos los días", "She kept at practicing piano every day"),
    ("Siguió con sus estudios a pesar de las dificultades", "He kept at his studies despite the difficulties"),

    # Keep away
    ("Mantén alejados a los perros del jardín", "Keep the dogs away from the garden"),
    ("Tienen un sistema de seguridad para mantener alejados a los intrusos", "They have a security system to keep away intruders"),
    ("Mantén esos químicos lejos de los niños", "Keep those chemicals away from the children"),

    # Keep back
    ("Los guardias mantuvieron a la multitud a distancia", "The guards kept back the crowd"),
    ("Mantén a los espectadores lejos del escenario", "Keep the spectators back from the stage"),
    ("El entrenador mantuvo a los jugadores alejados de los reporteros", "The coach kept the players back from the reporters"),

    # Keep down
    ("Por favor, mantén el volumen bajo", "Please, keep down the volume"),
    ("Debemos mantener bajos los costos", "We need to keep down the costs"),
    ("Mantén baja tu voz en la biblioteca", "Keep your voice down in the library"),

    # Keep from
    ("No pude evitar reírme durante la reunión", "I couldn't keep myself from laughing during the meeting"),
    ("Se esforzaba por no comer dulces", "She tried hard to keep from eating sweets"),
    ("No puedo evitar preocuparme por él", "I can't keep myself from worrying about him"),

    # Keep in
    ("Mantenían a los animales dentro durante la noche", "They kept the animals in at night"),
    ("Tuvo que quedarse en casa debido a la enfermedad", "He had to keep in due to the illness"),
    ("Mantén los documentos importantes en la caja fuerte", "Keep the important documents in the safe"),

    # Keep off
    ("Evita hablar de política en la cena", "Keep off politics at the dinner"),
    ("El profesor evitó el tema controvertido en clase", "The teacher kept off the controversial topic in class"),
    ("Intenta no hablar de tus problemas personales en el trabajo", "Try to keep off your personal issues at work"),

    # Keep on
    ("Sigue caminando, casi hemos llegado", "Keep on walking, we are almost there"),
    ("Ella sigue hablando incluso cuando nadie la escucha", "She keeps on talking even when no one is listening"),
    ("No dejes de creer en tus sueños", "Don't keep on believing in your dreams"),

    # Keep out
    ("Mantenemos fuera a los extraños de la propiedad", "We keep strangers out of the property"),
    ("El cartel dice 'Mantener fuera a los no autorizados'", "The sign says 'Keep out unauthorized persons'"),
    ("Mantén a los niños fuera de la cocina cuando cocinas", "Keep the children out of the kitchen when cooking"),

    # Keep to
    ("Sigamos con el horario original", "Let's keep to the original schedule"),
    ("Es importante mantenerse firme en tus principios", "It's important to keep to your principles"),
    ("Mantengamos el plan como está", "Let's keep to the plan as it is"),

    # Keep up (second instance)
    ("Mantén el buen rendimiento en la escuela", "Keep up the good performance at school"),
    ("Ella le dijo a su hijo que mantuviera el buen trabajo", "She told her son to keep up the good work")


]

# Preparar los datos para un archivo .csv
import csv

# Ruta del archivo CSV
csv_file_path = './datasets/mini_pool_phrasal_verbs_examples.csv'

# Crear y escribir en el archivo CSV
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter='~')
    for par in ejemplos:
        writer.writerow(par)

csv_file_path


'./datasets/mini_pool_phrasal_verbs_examples.csv'

# Clasificador gramatical de frases

In [ ]:
import re

# Funciones para identificar las categorías gramaticales
def is_simple_to_be(sentence):
    return re.search(r'\bis\b|\bam\b|\bare\b', sentence, re.IGNORECASE) is not None

def is_future_will_going_to(sentence):
    return re.search(r'\bwill\b|\bgoing to\b', sentence, re.IGNORECASE) is not None

def is_past_perfect(sentence):
    return re.search(r'\bhad\b', sentence, re.IGNORECASE) is not None

def is_possessive_pronouns(sentence):
    return re.search(r'\bmy\b|\byour\b|\bhis\b|\bher\b|\bits\b|\bour\b|\btheir\b', sentence, re.IGNORECASE) is not None

def is_there_to_be(sentence):
    return re.search(r'\bthere (is|are)\b', sentence, re.IGNORECASE) is not None

def is_auxiliary_verbs(sentence):
    return re.search(r'\bdo\b|\bdoes\b|\bdid\b|\bhave\b|\bhas\b|\bhad\b', sentence, re.IGNORECASE) is not None

def is_comparatives_superlatives(sentence):
    return re.search(r'\ber\b|\best\b', sentence, re.IGNORECASE) is not None  # Basic check for -er, -est endings

def is_imperatives(sentence):
    return sentence.endswith('!')  # Basic check for imperatives

def is_the_accusative(sentence):
    # This is complex to detect with simple regex, so we'll leave it as False
    return False

def is_modals(sentence):
    return re.search(r'\bcan\b|\bcould\b|\bmay\b|\bmight\b|\bmust\b|\bshall\b|\bshould\b|\bwill\b|\bwould\b', sentence, re.IGNORECASE) is not None

def is_too_much_many_enough(sentence):
    return re.search(r'\btoo much\b|\btoo many\b|\benough\b', sentence, re.IGNORECASE) is not None

def is_used_to_get_used_to(sentence):
    return re.search(r'\bused to\b|\bget used to\b', sentence, re.IGNORECASE) is not None

def is_countable_uncountable(sentence):
    # This is complex to detect with simple regex, so we'll leave it as False
    return False

def is_irregular_verbs(sentence):
    # Detecting irregular verbs would require a comprehensive list, so we'll leave it as False
    return False

# Actualizar el dataframe según las funciones de detección
for index, row in csv_data.iterrows():
    csv_data.at[index, 'simple To Be'] = is_simple_to_be(row['English'])
    csv_data.at[index, 'Future Will/be going to'] = is_future_will_going_to(row['English'])
    csv_data.at[index, 'Past Perfect'] = is_past_perfect(row['English'])
    csv_data.at[index, 'possesive pronouns'] = is_possessive_pronouns(row['English'])
    csv_data.at[index, 'there + to be'] = is_there_to_be(row['English'])
    csv_data.at[index, 'Auxiliary verbs'] = is_auxiliary_verbs(row['English'])
    csv_data.at[index, 'Comparatives & Superlatives'] = is_comparatives_superlatives(row['English'])
    csv_data.at[index, 'Direct & Indirect imperatives'] = is_imperatives(row['English'])
    csv_data.at[index, 'The accusative'] = is_the_accusative(row['English'])
    csv_data.at[index, 'Modals'] = is_modals(row['English'])
    csv_data.at[index, 'Too much-many/enough'] = is_too_much_many_enough(row['English'])
    csv_data.at[index, 'Used to/ to be-get used to'] = is_used_to_get_used_to(row['English'])
    csv_data.at[index, 'contables & uncontables'] = is_countable_uncountable(row['English'])
    csv_data.at[index, 'Irregular Verbs'] = is_irregular_verbs(row['English'])

# Guardar el dataframe actualizado en formato CSV
updated_csv_labeled_path = './datasets/Traslate_spanish_english_etiquetado_actualizado.csv'
csv_data.to_csv(updated_csv_labeled_path, sep='~', index=False)

updated_csv_labeled_path


# PRUEBAS VARIAS

In [55]:
#incluye carpeta media en .gitinore
!echo "media" >> .gitignore

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)


Cambiar el pitch:

In [39]:
import librosa
from scipy.io.wavfile import write

# Cargar el archivo de audio
y, sr = librosa.load('tortoise-tts/tortoise/voices/javier/audiotortoise2.wav', sr=None)

# Cambiar el tono del audio sin alterar la duración
y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=1)  # Aumentar el tono #n_steps=-1 mas grave #n_steps=1 mas agudo

# Guardar el audio modificado
write('shifted_output.wav', sr, y_shifted.astype('float32'))


Cambiar escpectograma

In [69]:
import pyworld as pw
import numpy as np
import librosa
import soundfile as sf

# Cargar el archivo de audio
file_path = "tortoise-tts/tortoise/voices/javier/audiotortoise2.wav"
x, fs = librosa.load(file_path, sr=22050)

# Descomposición de WORLD
f0, sp, ap = pw.wav2world(x.astype('double'), fs)

# Aumentar la frecuencia fundamental para feminizar la voz
f0[f0>0] *= 1.8

# Usar el manipulador de formantes de WORLD para modificar los formantes
# sp = pw.cheaptrick(x.astype('double'), f0, np.linspace(0, 0.5, len(x), endpoint=False), fs)

# Reconstruir la señal de audio con los componentes modificados
y = pw.synthesize(f0, sp, ap, fs)

# Guardar la señal de audio modificada
sf.write('voice_feminized.wav', y, fs)


Reverberación

In [47]:
import librosa
from scipy.signal import convolve
import numpy as np
import soundfile as sf

# Carga tu archivo de audio
y, sr = librosa.load('tortoise-tts/tortoise/voices/javier/audiotortoise2.wav', sr=None)

# Crea una respuesta al impulso simple para la reverberación (esto es bastante básico y podrías querer usar una respuesta al impulso más realista)
def simple_reverb_ir(duration, sr, decay=0.005):
    t = np.linspace(0, duration, int(sr * duration))
    ir = np.exp(-decay * t)
    return ir

# Crea una respuesta al impulso y convolútala con tu señal de audio
ir = simple_reverb_ir(0.1, sr)
y_reverb = convolve(y, ir)

# Normalizar el audio para evitar clipping
y_reverb = y_reverb / np.max(np.abs(y_reverb), axis=0)

# Guarda el audio resultante
sf.write('output_with_reverb.wav', y_reverb, sr)


In [ ]:
# antiguo codigo para juntar lineas del .vtt para traudcir
import re

def traducir(tipo_traductor, text, origen="eng", destino="spa"):
    if tipo_traductor == "helsinki":
        if origen == "eng":
            translated_text = translate_text_to_spanish(text)
        else:
            translated_text = translate_text_to_english(text)
        return translated_text

def traducir_subtitulos(tipo_traductor, input_file, output_file, origen="eng", destino="spa"):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        sentence = ""
        primera_linea = True
        max_paquetes = 4 #número de líneas que se agruparán para traducir
        paquete_count = 0
        buffer_traslator = []
        buffer_cabeceras = []
        agrupando = False
        for line in infile:
            if primera_linea:
                outfile.write(line) #WEBVTT
                primera_linea = False
            es_cabecera_temporal = re.match(r'^\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}\.\d{3}', line)
            if es_cabecera_temporal and paquete_count == 0:
                agrupando = True
                buffer_cabeceras = []
                # outfile.write(line)
            if agrupando:
                if es_cabecera_temporal:
                    buffer_cabeceras.append(line)
                if not es_cabecera_temporal and line.strip() != "":
                    paquete_count += 1
                    buffer_traslator.append(line.strip() + " $")
                    if paquete_count == max_paquetes:
                        #hemos terminado de agrupar ahora toca traducir y después
                        paquete_count = 0
                        agrupando = False
                        print("\nbuffer_traslator:", "".join(buffer_traslator))
                        translated_sentence = traducir(tipo_traductor, "".join(buffer_traslator), origen, destino)
                        buffer_traslator = []
                        print("translated_sentence:", translated_sentence, "\n")
                        # quitar último $
                        ultimo_dolar_index = translated_sentence.rfind("$")
                        translated_sentence = translated_sentence[:ultimo_dolar_index]

                        # translated_sentence = translated_sentence.replace(" $", " ") 

                        # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
                        translated_sentence_list = translated_sentence.split("$")
                        print("translated_sentence_list:", translated_sentence_list)
                        for i in range(len(translated_sentence_list)):
                            outfile.write(buffer_cabeceras[i])
                            outfile.write(translated_sentence_list[i].strip() + "\n")
                            outfile.write("\n")
                        


        if paquete_count != 0: #hemos terminado pero se han quedado lienas sin escribir
            print("ULTIMO PAQUETE!!!")
            print("\nbuffer_traslator:", buffer_traslator)
            translated_sentence = traducir(tipo_traductor, "".join(buffer_traslator), origen, destino)
            buffer_traslator = []
            print("translated_sentence:", translated_sentence, "\n")
            # quitar último $
            ultimo_dolar_index = translated_sentence.rfind("$")
            translated_sentence = translated_sentence[:ultimo_dolar_index]

            # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
            translated_sentence_list = translated_sentence.split("$")
            print("translated_sentence_list:", translated_sentence_list)
            for i in range(len(translated_sentence_list)):
                outfile.write(buffer_cabeceras[i])
                outfile.write(translated_sentence_list[i].strip() + "\n")   

# Cosas

In [ ]:
def reemplazar_saltos_linea(lista_strings):
    """
    Esta función toma una lista de strings y reemplaza todos los
    saltos de línea (\n) por espacios en blanco en cada string.

    :param lista_strings: Lista de strings a procesar.
    :return: Lista de strings con los saltos de línea reemplazados por espacios.
    """
    return [s.replace("\n", " ") for s in lista_strings]


ejemplos_phrasal_verbs = reemplazar_saltos_linea(ejemplos_phrasal_verbs)

print(ejemplos_phrasal_verbs[:5])